In [81]:
import numpy as np
import pandas as pd

df = pd.read_csv("../../data/epoch_benchmark_data/deepresearchbench_external.csv")
df.columns

Index(['Model version', 'Average score', 'Release date', 'Organization',
       'Country', 'Training compute (FLOP)', 'Training compute notes', 'Agent',
       'Notes', 'Source', 'Source link', 'id'],
      dtype='object')

Index(['Model version', 'Average score', 'Release date', 'Organization',
       'Country', 'Training compute (FLOP)', 'Training compute notes', 'Agent',
       'Notes', 'Source', 'Source link', 'id'],
      dtype='object')

In [66]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import io
# Neon color palette (bright colors for contrast on dark bg)
NEON_PALETTE = ['#FF00FF', '#00FFFF', '#FFFF00', '#FF0080', '#00FF80', '#8000FF', '#FF8000', '#00FF00']
def prepare_data(df):
    """
    Prepare the DataFrame for plotting.
    """
    df = df.copy()
    if 'Release date' in df.columns:
        # More robust date parsing: try multiple formats
        date_formats = ['%Y-%m-%d', '%m/%d/%Y', '%Y/%m/%d', '%d/%m/%Y']
        for fmt in date_formats:
            df['Release date'] = pd.to_datetime(df['Release date'], format=fmt, errors='coerce')
        # If still NaT, try infer
        df['Release date'] = pd.to_datetime(df['Release date'], errors='coerce', infer_datetime_format=True)
    # Handle percentages: assume decimal, convert to 0-100
    percent_cols = ['Unguided % Solved', 'Subtask-Guided % Solved', 'Subtasks % Solved']
    for col in percent_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce') * 100
            df[col] = df[col].fillna(0)
    # Handle times: numeric, fill NaN with large value or 0? Use np.nan for now, but for plots dropna
    time_cols = ['Unguided most difficult task solved (minutes)', 'Subtask-Guided most difficult task solved (minutes)']
    for col in time_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            df[col] = df[col].fillna(np.nan)
    # Handle Training compute (FLOP): convert to numeric, leave NaN as NaN (to avoid log issues)
    if 'Training compute (FLOP)' in df.columns:
        df['Training compute (FLOP)'] = pd.to_numeric(df['Training compute (FLOP)'], errors='coerce')
    if 'Model version' in df.columns and 'Name' in df.columns:
        df['Model version'] = df['Model version'].fillna(df['Name'])
    return df
def plot_unguided_over_time(df, output_file=None):
    """
    Line plot: Model Unguided % Solved Evolution by Release Date (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Release date', 'Unguided % Solved'])
    if df_plot.empty:
        print("No valid data for plot_unguided_over_time")
        return
    df_plot = df_plot.sort_values('Release date')
    fig = px.line(df_plot, x='Release date', y='Unguided % Solved', color='Organization',
                  title='Cybench External Unguided % Solved Trends Over Time', markers=True,
                  color_discrete_sequence=NEON_PALETTE)
    fig.update_xaxes(type='date', showgrid=True, gridwidth=1, gridcolor='#333333')
    fig.update_yaxes(range=[0, 100], showgrid=True, gridwidth=1, gridcolor='#333333')
    fig.update_traces(line=dict(width=2), marker=dict(size=6))
    fig.update_layout(template='plotly_dark', font=dict(color='white'), 
                      height=600, width=1000) # Dark theme + white text + adjusted size
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_bar_by_org(df, output_file=None):
    """
    Bar plot: Average Unguided % Solved by Organization (dark theme), sorted descending.
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Unguided % Solved'])
    if df_plot.empty:
        print("No valid data for plot_bar_by_org")
        return
    df_grouped = df_plot.groupby('Organization')['Unguided % Solved'].mean().reset_index()
    df_grouped = df_grouped.sort_values('Unguided % Solved', ascending=False)
    fig = px.bar(df_grouped, x='Unguided % Solved', y='Organization', orientation='h',
                 title='Average Cybench External Unguided % Solved by Organization',
                 color_discrete_sequence=NEON_PALETTE)
    fig.update_xaxes(range=[0, 100], showgrid=True, gridwidth=1, gridcolor='#333333')
    fig.update_traces(marker=dict(line=dict(width=1, color='white')))
    fig.update_layout(template='plotly_dark', font=dict(color='white'),
                      height=600, width=1000) # Dark theme + white text + adjusted size
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_compute_growth(df, output_file=None):
    """
    Area plot: Cumulative Compute vs. Release Date (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Release date', 'Training compute (FLOP)'])
    df_plot = df_plot[df_plot['Training compute (FLOP)'] > 0]
    if df_plot.empty:
        print("No valid data for plot_compute_growth")
        return
    df_plot = df_plot.sort_values('Release date')
    fig = px.area(df_plot, x='Release date', y='Training compute (FLOP)', color='Country',
                  title='Model Compute Growth by Country',
                  color_discrete_sequence=NEON_PALETTE)
    fig.update_xaxes(type='date', showgrid=True, gridwidth=1, gridcolor='#333333')
    fig.update_yaxes(type='log', showgrid=True, gridwidth=1, gridcolor='#333333')
    fig.update_layout(template='plotly_dark', font=dict(color='white'),
                      height=600, width=1000)
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_unguided_distribution(df, output_file=None):
    """
    Box plot: Unguided % Solved Distribution by Country (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Unguided % Solved'])
    if df_plot.empty:
        print("No valid data for plot_unguided_distribution")
        return
    fig = px.box(df_plot, x='Country', y='Unguided % Solved', points='outliers',
                 title='Unguided % Solved Distributions by Country',
                 color_discrete_sequence=NEON_PALETTE)
    fig.update_yaxes(range=[0, 100], showgrid=True, gridwidth=1, gridcolor='#333333')
    fig.update_layout(template='plotly_dark', font=dict(color='white'),
                      height=600, width=1000)
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_scaling_curve(df, output_file=None):
    """
    Scatter plot: Unguided % Solved vs. Training Compute (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Unguided % Solved', 'Training compute (FLOP)'])
    df_plot = df_plot[df_plot['Training compute (FLOP)'] > 0]
    if df_plot.empty:
        print("No valid data for plot_scaling_curve")
        return
    fig = px.scatter(df_plot, x='Training compute (FLOP)', y='Unguided % Solved', color='Organization',
                     log_x=True, trendline='ols',
                     title='Unguided % Solved vs. Compute (Scaling Curve)',
                     color_discrete_sequence=NEON_PALETTE)
    fig.update_yaxes(range=[0, 100], showgrid=True, gridwidth=1, gridcolor='#333333')
    fig.update_xaxes(type='log', showgrid=True, gridwidth=1, gridcolor='#333333')
    fig.update_traces(marker=dict(size=8))
    fig.update_layout(template='plotly_dark', font=dict(color='white'),
                      height=600, width=1000)
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_unguided_bubbles(df, output_file=None):
    """
    Bubble chart: Unguided % Solved Performance vs. Release Date (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Release date', 'Unguided % Solved', 'Training compute (FLOP)'])
    df_plot = df_plot[df_plot['Training compute (FLOP)'] > 0]
    if df_plot.empty:
        print("No valid data for plot_unguided_bubbles")
        return
    df_plot = df_plot.sort_values('Release date')
    fig = px.scatter(df_plot, x='Release date', y='Unguided % Solved', size='Training compute (FLOP)',
                     color='Organization', hover_name='Name',
                     title='Cybench External Unguided % Solved Over Time (Bubble = Compute)',
                     color_discrete_sequence=NEON_PALETTE)
    fig.update_xaxes(type='date', showgrid=True, gridwidth=1, gridcolor='#333333')
    fig.update_yaxes(range=[0, 100], showgrid=True, gridwidth=1, gridcolor='#333333')
    fig.update_traces(marker=dict(sizemin=4, sizemode='area'))
    fig.update_layout(template='plotly_dark', font=dict(color='white'),
                      height=600, width=1000)
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_subtask_guided_over_time(df, output_file=None):
    """
    Line plot: Model Subtask-Guided % Solved Evolution by Release Date (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Release date', 'Subtask-Guided % Solved'])
    if df_plot.empty:
        print("No valid data for plot_subtask_guided_over_time")
        return
    df_plot = df_plot.sort_values('Release date')
    fig = px.line(df_plot, x='Release date', y='Subtask-Guided % Solved', color='Organization',
                  title='Cybench External Subtask-Guided % Solved Trends Over Time', markers=True,
                  color_discrete_sequence=NEON_PALETTE)
    fig.update_xaxes(type='date', showgrid=True, gridwidth=1, gridcolor='#333333')
    fig.update_yaxes(range=[0, 100], showgrid=True, gridwidth=1, gridcolor='#333333')
    fig.update_traces(line=dict(width=2), marker=dict(size=6))
    fig.update_layout(template='plotly_dark', font=dict(color='white'), 
                      height=600, width=1000) # Dark theme + white text + adjusted size
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_correlation_heatmap(df, output_file=None):
    """
    Heatmap: Correlation Matrix (dark theme with adjusted scale).
    """
    df_plot = prepare_data(df)
    corr_cols = ['Unguided % Solved', 'Subtask-Guided % Solved', 'Subtasks % Solved']
    df_plot_corr = df_plot.dropna(subset=corr_cols)
    if df_plot_corr.empty:
        print("No valid data for plot_correlation_heatmap")
        return
    corr_df = df_plot_corr[corr_cols].corr()
    fig = go.Figure(data=go.Heatmap(z=corr_df.values, x=corr_df.columns, y=corr_df.index,
                                    colorscale='Viridis', zmin=-1, zmax=1))
    fig.update_layout(template='plotly_dark', title='Cybench External Metrics Correlation Heatmap', font=dict(color='white'),
                      height=600, width=1000)
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_performance_hierarchy(df, output_file=None):
    """
    Sunburst plot: Hierarchical Breakdown (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Unguided % Solved', 'Country', 'Organization'])
    if df_plot.empty:
        print("No valid data for plot_performance_hierarchy")
        return
    org_scores = df_plot.groupby(['Country', 'Organization'])['Unguided % Solved'].mean().reset_index()
    fig = px.sunburst(org_scores, path=['Country', 'Organization'], values='Unguided % Solved',
                      title='Performance Hierarchy: Country > Org',
                      color_discrete_sequence=NEON_PALETTE)
    fig.update_layout(template='plotly_dark', font=dict(color='white'),
                      height=600, width=1000)
    if output_file:
        fig.write_html(output_file)
    fig.show()
def create_dashboard(df, output_file='cybench_external_dashboard.html'):
    """
    Combined dashboard (dark theme) with Markdown write-up inserted at the top.
    """
    df_plot = prepare_data(df)
    if df_plot.empty:
        print("No valid data for dashboard")
        return
    fig = make_subplots(rows=2, cols=2,
                        subplot_titles=('Unguided % Solved Over Time', 'Scaling Curve',
                                        'Bar by Org', 'Subtask-Guided % Solved Distribution by Country'),
                        specs=[
                            [{"secondary_y": False}, {"secondary_y": False}],
                            [{"secondary_y": False}, {"secondary_y": False}]
                        ])
    # Line plot for Unguided
    df_line = df_plot.dropna(subset=['Release date', 'Unguided % Solved']).sort_values('Release date')
    if not df_line.empty:
        line_fig = px.line(df_line, x='Release date', y='Unguided % Solved', color='Organization', markers=True,
                           color_discrete_sequence=NEON_PALETTE)
        for trace in line_fig.data:
            fig.add_trace(trace, row=1, col=1)
    # Scatter plot
    df_scatter = df_plot.dropna(subset=['Unguided % Solved', 'Training compute (FLOP)'])
    df_scatter = df_scatter[df_scatter['Training compute (FLOP)'] > 0]
    if not df_scatter.empty:
        scatter_fig = px.scatter(df_scatter, x='Training compute (FLOP)', y='Unguided % Solved', log_x=True,
                                 color_discrete_sequence=NEON_PALETTE)
        for trace in scatter_fig.data:
            fig.add_trace(trace, row=1, col=2)
    # Bar plot: Average Unguided % Solved by Org (horizontal)
    df_bar = df_plot.dropna(subset=['Unguided % Solved'])
    if not df_bar.empty:
        df_grouped_bar = df_bar.groupby('Organization')['Unguided % Solved'].mean().reset_index()
        df_grouped_bar = df_grouped_bar.sort_values('Unguided % Solved', ascending=False)
        bar_fig = px.bar(df_grouped_bar, x='Unguided % Solved', y='Organization', orientation='h',
                         color_discrete_sequence=NEON_PALETTE)
        for trace in bar_fig.data:
            fig.add_trace(trace, row=2, col=1)
    # Box plot: Subtask-Guided by Country
    df_box = df_plot.dropna(subset=['Subtask-Guided % Solved'])
    if not df_box.empty:
        box_fig = px.box(df_box, x='Country', y='Subtask-Guided % Solved', points='outliers',
                         color_discrete_sequence=NEON_PALETTE)
        for trace in box_fig.data:
            fig.add_trace(trace, row=2, col=2)
    fig.update_layout(height=800, width=1200, title_text="Cybench External Model Performance Dashboard (Dark Neon Theme)",
                      template='plotly_dark', font=dict(color='white'))
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#333333')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#333333', range=[0, 100])
   
    # Generate HTML string
    html_content = fig.to_html(full_html=True, include_plotlyjs='cdn')
   
    # Markdown write-up as HTML (dark-themed)
    writeup_html = '''
    <div style="padding: 20px; background-color: #1f1f1f; color: #ffffff; border-bottom: 2px solid #333333; font-family: Arial, sans-serif;">
        <h2 style="color: #00FFFF; margin-bottom: 15px;">Cybench External</h2>
        <p style="line-height: 1.6; font-size: 14px;">
            Cybench External is an external evaluation benchmark for assessing AI agents' cybersecurity capabilities, focusing on real-world penetration testing and defensive tasks.
            It includes a suite of challenging CTF-style problems, evaluated in unguided (autonomous exploration) and subtask-guided (step-by-step prompting) modes to measure planning, execution, and adaptability.
            Tasks span vulnerability discovery, exploitation, and mitigation, with metrics capturing success rates and efficiency in solving complex, multi-stage scenarios.
        </p>
        <p style="line-height: 1.6; font-size: 14px; margin-top: 10px;">
            Key metrics include <strong>Unguided % Solved</strong> for autonomous performance, <strong>Subtask-Guided % Solved</strong> for prompted efficacy, and <strong>Subtasks % Solved</strong> for granular progress. Time to solve the most difficult task highlights efficiency gaps, underscoring the need for robust agentic reasoning in high-stakes domains.
        </p>
    </div>
    '''
   
    # Improved insertion: after scripts, before plot div
    body_start = html_content.find('<body>')
    if body_start != -1:
        plot_div_class_pos = html_content.find('class="plotly-graph-div"', body_start)
        if plot_div_class_pos != -1:
            div_start = html_content.rfind('<div', body_start, plot_div_class_pos)
            if div_start != -1:
                full_html = html_content[:div_start] + writeup_html + html_content[div_start:]
            else:
                full_html = html_content
        else:
            full_html = html_content
    else:
        full_html = html_content
   
    # Write to file
    if output_file:
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(full_html)
   
    fig.show()
    print(f"Dashboard saved to {output_file} with Cybench External write-up rendered at the top.")
# Preview (as requested)
plot_unguided_over_time(df)
plot_subtask_guided_over_time(df)
create_dashboard(df)

/tmp/ipykernel_9243/1772677666.py:20: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.



/tmp/ipykernel_9243/1772677666.py:20: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.



/tmp/ipykernel_9243/1772677666.py:20: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.



Dashboard saved to cybench_external_dashboard.html with Cybench External write-up rendered at the top.


In [69]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import io
# Neon color palette (bright colors for contrast on dark bg)
NEON_PALETTE = ['#FF00FF', '#00FFFF', '#FFFF00', '#FF0080', '#00FF80', '#8000FF', '#FF8000', '#00FF00']
def prepare_data(df):
    """
    Prepare the DataFrame for plotting.
    """
    df = df.copy()
    if 'Release date' in df.columns:
        # More robust date parsing: try multiple formats
        date_formats = ['%Y-%m-%d', '%m/%d/%Y', '%Y/%m/%d', '%d/%m/%Y']
        for fmt in date_formats:
            df['Release date'] = pd.to_datetime(df['Release date'], format=fmt, errors='coerce')
        # If still NaT, try infer
        df['Release date'] = pd.to_datetime(df['Release date'], errors='coerce', infer_datetime_format=True)
    # Handle percentages: assume decimal, convert to 0-100
    percent_cols = ['Unguided % Solved', 'Subtask-Guided % Solved', 'Subtasks % Solved']
    for col in percent_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce') * 100
            df[col] = df[col].fillna(0)
    # Handle times: numeric, fill NaN with large value or 0? Use np.nan for now, but for plots dropna
    time_cols = ['Unguided most difficult task solved (minutes)', 'Subtask-Guided most difficult task solved (minutes)']
    for col in time_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            df[col] = df[col].fillna(np.nan)
    # Handle Training compute (FLOP): convert to numeric, leave NaN as NaN (to avoid log issues)
    if 'Training compute (FLOP)' in df.columns:
        df['Training compute (FLOP)'] = pd.to_numeric(df['Training compute (FLOP)'], errors='coerce')
    if 'Model version' in df.columns and 'Name' in df.columns:
        df['Model version'] = df['Model version'].fillna(df['Name'])
    return df
def plot_unguided_over_time(df, output_file=None):
    """
    Line plot: Model Unguided % Solved Evolution by Release Date (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Release date', 'Unguided % Solved'])
    if df_plot.empty:
        print("No valid data for plot_unguided_over_time")
        return
    df_plot = df_plot.sort_values('Release date')
    fig = px.line(df_plot, x='Release date', y='Unguided % Solved', color='Organization',
                  title='Cybench External Unguided % Solved Trends Over Time', markers=True,
                  color_discrete_sequence=NEON_PALETTE)
    fig.update_xaxes(type='date', showgrid=True, gridwidth=1, gridcolor='#333333')
    fig.update_yaxes(range=[0, 100], showgrid=True, gridwidth=1, gridcolor='#333333')
    fig.update_traces(line=dict(width=2), marker=dict(size=6))
    fig.update_layout(template='plotly_dark', font=dict(color='white'), 
                      height=600, width=1000) # Dark theme + white text + adjusted size
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_bar_by_org(df, output_file=None):
    """
    Bar plot: Average Unguided % Solved by Organization (dark theme), sorted descending.
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Unguided % Solved'])
    if df_plot.empty:
        print("No valid data for plot_bar_by_org")
        return
    df_grouped = df_plot.groupby('Organization')['Unguided % Solved'].mean().reset_index()
    df_grouped = df_grouped.sort_values('Unguided % Solved', ascending=False)
    fig = px.bar(df_grouped, x='Unguided % Solved', y='Organization', orientation='h',
                 title='Average Cybench External Unguided % Solved by Organization',
                 color_discrete_sequence=NEON_PALETTE)
    fig.update_xaxes(range=[0, 100], showgrid=True, gridwidth=1, gridcolor='#333333')
    fig.update_traces(marker=dict(line=dict(width=1, color='white')))
    fig.update_layout(template='plotly_dark', font=dict(color='white'),
                      height=600, width=1000) # Dark theme + white text + adjusted size
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_compute_growth(df, output_file=None):
    """
    Area plot: Cumulative Compute vs. Release Date (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Release date', 'Training compute (FLOP)'])
    df_plot = df_plot[df_plot['Training compute (FLOP)'] > 0]
    if df_plot.empty:
        print("No valid data for plot_compute_growth")
        return
    df_plot = df_plot.sort_values('Release date')
    fig = px.area(df_plot, x='Release date', y='Training compute (FLOP)', color='Country',
                  title='Model Compute Growth by Country',
                  color_discrete_sequence=NEON_PALETTE)
    fig.update_xaxes(type='date', showgrid=True, gridwidth=1, gridcolor='#333333')
    fig.update_yaxes(type='log', showgrid=True, gridwidth=1, gridcolor='#333333')
    fig.update_layout(template='plotly_dark', font=dict(color='white'),
                      height=600, width=1000)
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_unguided_distribution(df, output_file=None):
    """
    Violin plot: Unguided % Solved Distribution by Country (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Unguided % Solved'])
    if df_plot.empty:
        print("No valid data for plot_unguided_distribution")
        return
    fig = px.violin(df_plot, x='Country', y='Unguided % Solved', points='outliers', box=True,
                    title='Unguided % Solved Distributions by Country',
                    color_discrete_sequence=NEON_PALETTE)
    fig.update_yaxes(range=[0, 100], showgrid=True, gridwidth=1, gridcolor='#333333')
    fig.update_layout(template='plotly_dark', font=dict(color='white'),
                      height=600, width=1000)
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_scaling_curve(df, output_file=None):
    """
    Scatter plot: Unguided % Solved vs. Training Compute (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Unguided % Solved', 'Training compute (FLOP)'])
    df_plot = df_plot[df_plot['Training compute (FLOP)'] > 0]
    if df_plot.empty:
        print("No valid data for plot_scaling_curve")
        return
    fig = px.scatter(df_plot, x='Training compute (FLOP)', y='Unguided % Solved', color='Organization',
                     log_x=True, trendline='ols',
                     title='Unguided % Solved vs. Compute (Scaling Curve)',
                     color_discrete_sequence=NEON_PALETTE)
    fig.update_yaxes(range=[0, 100], showgrid=True, gridwidth=1, gridcolor='#333333')
    fig.update_xaxes(type='log', showgrid=True, gridwidth=1, gridcolor='#333333')
    fig.update_traces(marker=dict(size=8))
    fig.update_layout(template='plotly_dark', font=dict(color='white'),
                      height=600, width=1000)
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_unguided_bubbles(df, output_file=None):
    """
    Bubble chart: Unguided % Solved Performance vs. Release Date (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Release date', 'Unguided % Solved', 'Training compute (FLOP)'])
    df_plot = df_plot[df_plot['Training compute (FLOP)'] > 0]
    if df_plot.empty:
        print("No valid data for plot_unguided_bubbles")
        return
    df_plot = df_plot.sort_values('Release date')
    fig = px.scatter(df_plot, x='Release date', y='Unguided % Solved', size='Training compute (FLOP)',
                     color='Organization', hover_name='Name',
                     title='Cybench External Unguided % Solved Over Time (Bubble = Compute)',
                     color_discrete_sequence=NEON_PALETTE)
    fig.update_xaxes(type='date', showgrid=True, gridwidth=1, gridcolor='#333333')
    fig.update_yaxes(range=[0, 100], showgrid=True, gridwidth=1, gridcolor='#333333')
    fig.update_traces(marker=dict(sizemin=4, sizemode='area'))
    fig.update_layout(template='plotly_dark', font=dict(color='white'),
                      height=600, width=1000)
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_subtask_guided_over_time(df, output_file=None):
    """
    Line plot: Model Subtask-Guided % Solved Evolution by Release Date (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Release date', 'Subtask-Guided % Solved'])
    if df_plot.empty:
        print("No valid data for plot_subtask_guided_over_time")
        return
    df_plot = df_plot.sort_values('Release date')
    fig = px.line(df_plot, x='Release date', y='Subtask-Guided % Solved', color='Organization',
                  title='Cybench External Subtask-Guided % Solved Trends Over Time', markers=True,
                  color_discrete_sequence=NEON_PALETTE)
    fig.update_xaxes(type='date', showgrid=True, gridwidth=1, gridcolor='#333333')
    fig.update_yaxes(range=[0, 100], showgrid=True, gridwidth=1, gridcolor='#333333')
    fig.update_traces(line=dict(width=2), marker=dict(size=6))
    fig.update_layout(template='plotly_dark', font=dict(color='white'), 
                      height=600, width=1000) # Dark theme + white text + adjusted size
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_correlation_heatmap(df, output_file=None):
    """
    Heatmap: Correlation Matrix (dark theme with adjusted scale).
    """
    df_plot = prepare_data(df)
    corr_cols = ['Unguided % Solved', 'Subtask-Guided % Solved', 'Subtasks % Solved']
    df_plot_corr = df_plot.dropna(subset=corr_cols)
    if df_plot_corr.empty:
        print("No valid data for plot_correlation_heatmap")
        return
    corr_df = df_plot_corr[corr_cols].corr()
    fig = go.Figure(data=go.Heatmap(z=corr_df.values, x=corr_df.columns, y=corr_df.index,
                                    colorscale='Viridis', zmin=-1, zmax=1))
    fig.update_layout(template='plotly_dark', title='Cybench External Metrics Correlation Heatmap', font=dict(color='white'),
                      height=600, width=1000)
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_performance_hierarchy(df, output_file=None):
    """
    Sunburst plot: Hierarchical Breakdown (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Unguided % Solved', 'Country', 'Organization'])
    if df_plot.empty:
        print("No valid data for plot_performance_hierarchy")
        return
    org_scores = df_plot.groupby(['Country', 'Organization'])['Unguided % Solved'].mean().reset_index()
    fig = px.sunburst(org_scores, path=['Country', 'Organization'], values='Unguided % Solved',
                      title='Performance Hierarchy: Country > Org',
                      color_discrete_sequence=NEON_PALETTE)
    fig.update_layout(template='plotly_dark', font=dict(color='white'),
                      height=600, width=1000)
    if output_file:
        fig.write_html(output_file)
    fig.show()
def create_dashboard(df, output_file='cybench_external_dashboard.html'):
    """
    Combined dashboard (dark theme) with Markdown write-up inserted at the top.
    """
    df_plot = prepare_data(df)
    if df_plot.empty:
        print("No valid data for dashboard")
        return
    fig = make_subplots(rows=2, cols=2,
                        subplot_titles=('Unguided % Solved Over Time', 'Scaling Curve',
                                        'Bar by Org', 'Subtask-Guided % Solved Distribution by Country (Violin)'),
                        specs=[
                            [{"secondary_y": False}, {"secondary_y": False}],
                            [{"secondary_y": False}, {"secondary_y": False}]
                        ])
    # Line plot for Unguided
    df_line = df_plot.dropna(subset=['Release date', 'Unguided % Solved']).sort_values('Release date')
    if not df_line.empty:
        line_fig = px.line(df_line, x='Release date', y='Unguided % Solved', color='Organization', markers=True,
                           color_discrete_sequence=NEON_PALETTE)
        for trace in line_fig.data:
            fig.add_trace(trace, row=1, col=1)
    # Scatter plot
    df_scatter = df_plot.dropna(subset=['Unguided % Solved', 'Training compute (FLOP)'])
    df_scatter = df_scatter[df_scatter['Training compute (FLOP)'] > 0]
    if not df_scatter.empty:
        scatter_fig = px.scatter(df_scatter, x='Training compute (FLOP)', y='Unguided % Solved', log_x=True,
                                 color_discrete_sequence=NEON_PALETTE)
        for trace in scatter_fig.data:
            fig.add_trace(trace, row=1, col=2)
    # Bar plot: Average Unguided % Solved by Org (horizontal)
    df_bar = df_plot.dropna(subset=['Unguided % Solved'])
    if not df_bar.empty:
        df_grouped_bar = df_bar.groupby('Organization')['Unguided % Solved'].mean().reset_index()
        df_grouped_bar = df_grouped_bar.sort_values('Unguided % Solved', ascending=False)
        bar_fig = px.bar(df_grouped_bar, x='Unguided % Solved', y='Organization', orientation='h',
                         color_discrete_sequence=NEON_PALETTE)
        for trace in bar_fig.data:
            fig.add_trace(trace, row=2, col=1)
    # Violin plot: Subtask-Guided by Country
    df_violin = df_plot.dropna(subset=['Subtask-Guided % Solved'])
    if not df_violin.empty:
        violin_fig = px.violin(df_violin, x='Country', y='Subtask-Guided % Solved', points='outliers', box=True,
                               color_discrete_sequence=NEON_PALETTE)
        for trace in violin_fig.data:
            fig.add_trace(trace, row=2, col=2)
    fig.update_layout(height=800, width=1200, title_text="Cybench External Model Performance Dashboard (Dark Neon Theme)",
                      template='plotly_dark', font=dict(color='white'))
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#333333')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#333333', range=[0, 100])
   
    # Generate HTML string
    html_content = fig.to_html(full_html=True, include_plotlyjs='cdn')
   
    # Markdown write-up as HTML (dark-themed)
    writeup_html = '''
    <div style="padding: 20px; background-color: #1f1f1f; color: #ffffff; border-bottom: 2px solid #333333; font-family: Arial, sans-serif;">
        <h2 style="color: #00FFFF; margin-bottom: 15px;">Cybench External</h2>
        <p style="line-height: 1.6; font-size: 14px;">
            Cybench External is an external evaluation benchmark for assessing AI agents' cybersecurity capabilities, focusing on real-world penetration testing and defensive tasks.
            It includes a suite of challenging CTF-style problems, evaluated in unguided (autonomous exploration) and subtask-guided (step-by-step prompting) modes to measure planning, execution, and adaptability.
            Tasks span vulnerability discovery, exploitation, and mitigation, with metrics capturing success rates and efficiency in solving complex, multi-stage scenarios.
        </p>
        <p style="line-height: 1.6; font-size: 14px; margin-top: 10px;">
            Key metrics include <strong>Unguided % Solved</strong> for autonomous performance, <strong>Subtask-Guided % Solved</strong> for prompted efficacy, and <strong>Subtasks % Solved</strong> for granular progress. Time to solve the most difficult task highlights efficiency gaps, underscoring the need for robust agentic reasoning in high-stakes domains.
        </p>
    </div>
    '''
   
    # Improved insertion: after scripts, before plot div
    body_start = html_content.find('<body>')
    if body_start != -1:
        plot_div_class_pos = html_content.find('class="plotly-graph-div"', body_start)
        if plot_div_class_pos != -1:
            div_start = html_content.rfind('<div', body_start, plot_div_class_pos)
            if div_start != -1:
                full_html = html_content[:div_start] + writeup_html + html_content[div_start:]
            else:
                full_html = html_content
        else:
            full_html = html_content
    else:
        full_html = html_content
   
    # Write to file
    if output_file:
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(full_html)
   
    fig.show()
    print(f"Dashboard saved to {output_file} with Cybench External write-up rendered at the top.")
# Preview (as requested)
plot_unguided_over_time(df)
plot_subtask_guided_over_time(df)
create_dashboard(df)

/tmp/ipykernel_9243/1439121824.py:20: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.



/tmp/ipykernel_9243/1439121824.py:20: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.



/tmp/ipykernel_9243/1439121824.py:20: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.



Dashboard saved to cybench_external_dashboard.html with Cybench External write-up rendered at the top.


In [51]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import io
# Neon color palette (bright colors for contrast on dark bg)
NEON_PALETTE = ['#FF00FF', '#00FFFF', '#FFFF00', '#FF0080', '#00FF80', '#8000FF', '#FF8000', '#00FF00']
def prepare_data(df):
    """
    Prepare the DataFrame for plotting.
    """
    df = df.copy()
    if 'Release date' in df.columns:
        # More robust date parsing: try multiple formats
        date_formats = ['%Y-%m-%d', '%m/%d/%Y', '%Y/%m/%d', '%d/%m/%Y']
        for fmt in date_formats:
            df['Release date'] = pd.to_datetime(df['Release date'], format=fmt, errors='coerce')
        # If still NaT, try infer
        df['Release date'] = pd.to_datetime(df['Release date'], errors='coerce', infer_datetime_format=True)
    # Handle all relevant numeric columns
    percent_cols = ['Overall pass (%)', 'SCAD Gen (%)', 'Render (%)', 'Chamfer Pass (%)', 'Haus. Pass', 'Vol. Pass']
    for col in percent_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            df[col] = df[col].fillna(0) * 100  # Convert decimal to percentage (0-100)
    other_numeric_cols = ['Total tasks', 'Total Passes', 'SCAD Gen Success', 'Render Successes', 'Checks Run', 'Avg Chamfer (mm)', 'Avg Haus. 95p (mm)']
    for col in other_numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            if col.startswith('Avg'):
                df[col] = df[col].fillna(np.nan)  # Leave averages as NaN if missing
            else:
                df[col] = df[col].fillna(0)
    # Handle Training compute (FLOP): convert to numeric, leave NaN as NaN (to avoid log issues)
    if 'Training compute (FLOP)' in df.columns:
        df['Training compute (FLOP)'] = pd.to_numeric(df['Training compute (FLOP)'], errors='coerce')
    if 'Model version' in df.columns and 'Name' in df.columns:
        df['Model version'] = df['Model version'].fillna(df['Name'])
    return df
def plot_overall_pass_over_time(df, output_file=None):
    """
    Line plot: Model Overall Pass Evolution by Release Date (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Release date', 'Overall pass (%)'])
    if df_plot.empty:
        print("No valid data for plot_overall_pass_over_time")
        return
    df_plot = df_plot.sort_values('Release date')
    fig = px.line(df_plot, x='Release date', y='Overall pass (%)', color='Organization',
                  title='CAD Eval External Overall Pass Trends Over Time', markers=True,
                  color_discrete_sequence=NEON_PALETTE)
    fig.update_xaxes(type='date', showgrid=True, gridwidth=1, gridcolor='#333333')
    fig.update_yaxes(range=[0, 100], showgrid=True, gridwidth=1, gridcolor='#333333')
    fig.update_traces(line=dict(width=2), marker=dict(size=6))
    fig.update_layout(template='plotly_dark', font=dict(color='white'), 
                      height=600, width=1000) # Dark theme + white text + adjusted size
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_bar_by_org(df, output_file=None):
    """
    Bar plot: Average Overall Pass by Organization (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Overall pass (%)'])
    if df_plot.empty:
        print("No valid data for plot_bar_by_org")
        return
    df_grouped = df_plot.groupby('Organization')['Overall pass (%)'].mean().reset_index()
    fig = px.bar(df_grouped, x='Organization', y='Overall pass (%)',
                 title='Average CAD Eval External Overall Pass by Organization',
                 color_discrete_sequence=NEON_PALETTE)
    fig.update_yaxes(range=[0, 100], showgrid=True, gridwidth=1, gridcolor='#333333')
    fig.update_traces(marker=dict(line=dict(width=1, color='white')))
    fig.update_layout(template='plotly_dark', font=dict(color='white'),
                      height=600, width=1000) # Dark theme + white text + adjusted size
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_compute_growth(df, output_file=None):
    """
    Area plot: Cumulative Compute vs. Release Date (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Release date', 'Training compute (FLOP)'])
    df_plot = df_plot[df_plot['Training compute (FLOP)'] > 0]
    if df_plot.empty:
        print("No valid data for plot_compute_growth")
        return
    df_plot = df_plot.sort_values('Release date')
    fig = px.area(df_plot, x='Release date', y='Training compute (FLOP)', color='Country',
                  title='Model Compute Growth by Country',
                  color_discrete_sequence=NEON_PALETTE)
    fig.update_xaxes(type='date', showgrid=True, gridwidth=1, gridcolor='#333333')
    fig.update_yaxes(type='log', showgrid=True, gridwidth=1, gridcolor='#333333')
    fig.update_layout(template='plotly_dark', font=dict(color='white'),
                      height=600, width=1000)
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_overall_pass_distribution(df, output_file=None):
    """
    Box plot: Overall Pass Distribution by Country (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Overall pass (%)'])
    if df_plot.empty:
        print("No valid data for plot_overall_pass_distribution")
        return
    fig = px.box(df_plot, x='Country', y='Overall pass (%)', points='outliers',
                 title='Overall Pass Distributions by Country',
                 color_discrete_sequence=NEON_PALETTE)
    fig.update_yaxes(range=[0, 100], showgrid=True, gridwidth=1, gridcolor='#333333')
    fig.update_layout(template='plotly_dark', font=dict(color='white'),
                      height=600, width=1000)
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_scaling_curve(df, output_file=None):
    """
    Scatter plot: Overall Pass vs. Training Compute (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Overall pass (%)', 'Training compute (FLOP)'])
    df_plot = df_plot[df_plot['Training compute (FLOP)'] > 0]
    if df_plot.empty:
        print("No valid data for plot_scaling_curve")
        return
    fig = px.scatter(df_plot, x='Training compute (FLOP)', y='Overall pass (%)', color='Prompt',
                     log_x=True, trendline='ols',
                     title='Overall Pass vs. Compute (Scaling Curve)',
                     color_discrete_sequence=NEON_PALETTE)
    fig.update_yaxes(range=[0, 100], showgrid=True, gridwidth=1, gridcolor='#333333')
    fig.update_xaxes(type='log', showgrid=True, gridwidth=1, gridcolor='#333333')
    fig.update_traces(marker=dict(size=8))
    fig.update_layout(template='plotly_dark', font=dict(color='white'),
                      height=600, width=1000)
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_overall_pass_bubbles(df, output_file=None):
    """
    Bubble chart: Overall Pass Performance vs. Release Date (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Release date', 'Overall pass (%)', 'Training compute (FLOP)'])
    df_plot = df_plot[df_plot['Training compute (FLOP)'] > 0]
    if df_plot.empty:
        print("No valid data for plot_overall_pass_bubbles")
        return
    df_plot = df_plot.sort_values('Release date')
    fig = px.scatter(df_plot, x='Release date', y='Overall pass (%)', size='Training compute (FLOP)',
                     color='Organization', hover_name='Name',
                     title='CAD Eval External Overall Pass Over Time (Bubble = Compute)',
                     color_discrete_sequence=NEON_PALETTE)
    fig.update_xaxes(type='date', showgrid=True, gridwidth=1, gridcolor='#333333')
    fig.update_yaxes(range=[0, 100], showgrid=True, gridwidth=1, gridcolor='#333333')
    fig.update_traces(marker=dict(sizemin=4, sizemode='area'))
    fig.update_layout(template='plotly_dark', font=dict(color='white'),
                      height=600, width=1000)
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_correlation_heatmap(df, output_file=None):
    """
    Heatmap: Correlation Matrix (dark theme with neon-inspired scale).
    """
    df_plot = prepare_data(df)
    corr_cols = ['Overall pass (%)', 'SCAD Gen (%)', 'Render (%)', 'Chamfer Pass (%)', 'Haus. Pass', 'Vol. Pass']
    df_plot_corr = df_plot.dropna(subset=corr_cols)
    if df_plot_corr.empty:
        print("No valid data for plot_correlation_heatmap")
        return
    corr_df = df_plot_corr[corr_cols].corr()
    fig = go.Figure(data=go.Heatmap(z=corr_df.values, x=corr_df.columns, y=corr_df.index,
                                    colorscale='Plasma'))
    fig.update_layout(template='plotly_dark', title='CAD Eval External Metrics Correlation Heatmap', font=dict(color='white'),
                      height=600, width=1000)
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_performance_hierarchy(df, output_file=None):
    """
    Sunburst plot: Hierarchical Breakdown (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Overall pass (%)', 'Country', 'Organization'])
    if df_plot.empty:
        print("No valid data for plot_performance_hierarchy")
        return
    org_scores = df_plot.groupby(['Country', 'Organization'])['Overall pass (%)'].mean().reset_index()
    fig = px.sunburst(org_scores, path=['Country', 'Organization'], values='Overall pass (%)',
                      title='Performance Hierarchy: Country > Org',
                      color_discrete_sequence=NEON_PALETTE)
    fig.update_layout(template='plotly_dark', font=dict(color='white'),
                      height=600, width=1000)
    if output_file:
        fig.write_html(output_file)
    fig.show()
def create_dashboard(df, output_file='cad_eval_external_dashboard.html'):
    """
    Combined dashboard (dark theme) with Markdown write-up inserted at the top.
    """
    df_plot = prepare_data(df)
    if df_plot.empty:
        print("No valid data for dashboard")
        return
    fig = make_subplots(rows=2, cols=2,
                        subplot_titles=('Overall Pass Over Time', 'Scaling Curve',
                                        'Bar by Org', 'Correlation Heatmap'),
                        specs=[
                            [{"secondary_y": False}, {"secondary_y": False}],
                            [{"secondary_y": False}, {"type": "heatmap"}]
                        ])
    # Line plot
    df_line = df_plot.dropna(subset=['Release date', 'Overall pass (%)']).sort_values('Release date')
    if not df_line.empty:
        line_fig = px.line(df_line, x='Release date', y='Overall pass (%)', color='Organization', markers=True,
                           color_discrete_sequence=NEON_PALETTE)
        for trace in line_fig.data:
            fig.add_trace(trace, row=1, col=1)
    # Scatter plot
    df_scatter = df_plot.dropna(subset=['Overall pass (%)', 'Training compute (FLOP)'])
    df_scatter = df_scatter[df_scatter['Training compute (FLOP)'] > 0]
    if not df_scatter.empty:
        scatter_fig = px.scatter(df_scatter, x='Training compute (FLOP)', y='Overall pass (%)', log_x=True,
                                 color='Prompt', color_discrete_sequence=NEON_PALETTE)
        for trace in scatter_fig.data:
            fig.add_trace(trace, row=1, col=2)
    # Bar plot: Average Overall Pass by Org
    df_bar = df_plot.dropna(subset=['Overall pass (%)'])
    if not df_bar.empty:
        df_grouped_bar = df_bar.groupby('Organization')['Overall pass (%)'].mean().reset_index()
        bar_fig = px.bar(df_grouped_bar, x='Organization', y='Overall pass (%)',
                         color_discrete_sequence=NEON_PALETTE)
        for trace in bar_fig.data:
            fig.add_trace(trace, row=2, col=1)
    # Heatmap
    corr_cols = ['Overall pass (%)', 'SCAD Gen (%)', 'Render (%)', 'Chamfer Pass (%)', 'Haus. Pass', 'Vol. Pass']
    df_heatmap = df_plot.dropna(subset=corr_cols)
    if not df_heatmap.empty:
        corr_df = df_heatmap[corr_cols].corr()
        heatmap = go.Heatmap(z=corr_df.values, x=corr_df.columns, y=corr_df.index, colorscale='Plasma')
        fig.add_trace(heatmap, row=2, col=2)
    fig.update_layout(height=800, width=1200, title_text="CAD Eval External Model Performance Dashboard (Dark Neon Theme)",
                      template='plotly_dark', font=dict(color='white'))
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#333333')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#333333', range=[0, 100])
   
    # Generate HTML string
    html_content = fig.to_html(full_html=True, include_plotlyjs='cdn')
   
    # Markdown write-up as HTML (dark-themed)
    writeup_html = '''
    <div style="padding: 20px; background-color: #1f1f1f; color: #ffffff; border-bottom: 2px solid #333333; font-family: Arial, sans-serif;">
        <h2 style="color: #00FFFF; margin-bottom: 15px;">Text to CAD Eval Benchmark</h2>
        <p style="line-height: 1.6; font-size: 14px;">
            The Text to CAD Eval benchmark assesses large language models (LLMs) on generating solid CAD models via OpenSCAD code from text prompts.
            It includes 25 tasks of varying complexity, from simple geometric shapes to multi-operation designs, each with a single correct interpretation for automated evaluation.
            Prompts are processed to produce OpenSCAD code, rendered to STL files, and compared against reference models using geometric metrics.
        </p>
        <p style="line-height: 1.6; font-size: 14px; margin-top: 10px;">
            Key metrics include <strong>Overall pass (%)</strong> for tasks passing all checks, <strong>SCAD Gen (%)</strong> for valid code generation, <strong>Render (%)</strong> for successful STL rendering, and geometric accuracy via <strong>Chamfer Pass (%)</strong>, Hausdorff distance, and volume differences.
            This benchmark reveals LLMs' proficiency in precise, sequential CAD instructions, with recent models showing marked improvements on complex tasks.
        </p>
    </div>
    '''
   
    # Improved insertion: after scripts, before plot div
    body_start = html_content.find('<body>')
    if body_start != -1:
        plot_div_class_pos = html_content.find('class="plotly-graph-div"', body_start)
        if plot_div_class_pos != -1:
            div_start = html_content.rfind('<div', body_start, plot_div_class_pos)
            if div_start != -1:
                full_html = html_content[:div_start] + writeup_html + html_content[div_start:]
            else:
                full_html = html_content
        else:
            full_html = html_content
    else:
        full_html = html_content
   
    # Write to file
    if output_file:
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(full_html)
   
    fig.show()
    print(f"Dashboard saved to {output_file} with CAD Eval External write-up rendered at the top.")
# Preview (as requested)
plot_overall_pass_over_time(df)
create_dashboard(df)

/tmp/ipykernel_9243/3946966021.py:20: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.



/tmp/ipykernel_9243/3946966021.py:20: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.



Dashboard saved to cad_eval_external_dashboard.html with CAD Eval External write-up rendered at the top.


In [46]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import io
# Neon color palette (bright colors for contrast on dark bg)
NEON_PALETTE = ['#FF00FF', '#00FFFF', '#FFFF00', '#FF0080', '#00FF80', '#8000FF', '#FF8000', '#00FF00']
def prepare_data(df):
    """
    Prepare the DataFrame for plotting.
    """
    df = df.copy()
    if 'Release date' in df.columns:
        df['Release date'] = pd.to_datetime(df['Release date'], errors='coerce')
    # Handle Score: fill NaN with 0
    numeric_cols = ['Score']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            df[col] = df[col].fillna(0)
    # Handle Training compute (FLOP): convert to numeric, leave NaN as NaN (to avoid log issues)
    if 'Training compute (FLOP)' in df.columns:
        df['Training compute (FLOP)'] = pd.to_numeric(df['Training compute (FLOP)'], errors='coerce')
    if 'Model version' in df.columns and 'Name' in df.columns:
        df['Model version'] = df['Model version'].fillna(df['Name'])
    return df
def plot_score_over_time(df, output_file=None):
    """
    Line plot: Model Score Evolution by Release Date (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Release date', 'Score'])
    df_plot = df_plot.sort_values('Release date')
    fig = px.line(df_plot, x='Release date', y='Score', color='Organization',
                  title='BoolQ Score Trends Over Time', markers=True,
                  color_discrete_sequence=NEON_PALETTE)
    fig.update_xaxes(type='date')
    fig.update_layout(template='plotly_dark', font=dict(color='white')) # Dark theme + white text
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_bar_by_org(df, output_file=None):
    """
    Bar plot: Average Score by Organization (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Score'])
    df_grouped = df_plot.groupby('Organization')['Score'].mean().reset_index()
    fig = px.bar(df_grouped, x='Organization', y='Score',
                 title='Average BoolQ Score by Organization',
                 color_discrete_sequence=NEON_PALETTE)
    fig.update_layout(template='plotly_dark', font=dict(color='white')) # Dark theme + white text
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_compute_growth(df, output_file=None):
    """
    Area plot: Cumulative Compute vs. Release Date (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Release date', 'Training compute (FLOP)'])
    df_plot = df_plot[df_plot['Training compute (FLOP)'] > 0]
    df_plot = df_plot.sort_values('Release date')
    fig = px.area(df_plot, x='Release date', y='Training compute (FLOP)', color='Country',
                  title='Model Compute Growth by Country',
                  color_discrete_sequence=NEON_PALETTE)
    fig.update_xaxes(type='date')
    fig.update_layout(template='plotly_dark', font=dict(color='white'))
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_score_distribution(df, output_file=None):
    """
    Box plot: Score Distribution by Country (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Score'])
    fig = px.box(df_plot, x='Country', y='Score', points='outliers',
                 title='Score Distributions by Country',
                 color_discrete_sequence=NEON_PALETTE)
    fig.update_layout(template='plotly_dark', font=dict(color='white'))
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_scaling_curve(df, output_file=None):
    """
    Scatter plot: Score vs. Training Compute (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Score', 'Training compute (FLOP)'])
    df_plot = df_plot[df_plot['Training compute (FLOP)'] > 0]
    fig = px.scatter(df_plot, x='Training compute (FLOP)', y='Score', color='Shots',
                     log_x=True, trendline='ols',
                     title='Score vs. Compute (Scaling Curve)',
                     color_discrete_sequence=NEON_PALETTE)
    fig.update_layout(template='plotly_dark', font=dict(color='white'))
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_score_bubbles(df, output_file=None):
    """
    Bubble chart: Score Performance vs. Release Date (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Release date', 'Score', 'Training compute (FLOP)'])
    df_plot = df_plot[df_plot['Training compute (FLOP)'] > 0]
    df_plot = df_plot.sort_values('Release date')
    fig = px.scatter(df_plot, x='Release date', y='Score', size='Training compute (FLOP)',
                     color='Organization', hover_name='Name',
                     title='BoolQ Scores Over Time (Bubble = Compute)',
                     color_discrete_sequence=NEON_PALETTE)
    fig.update_xaxes(type='date')
    fig.update_layout(template='plotly_dark', font=dict(color='white'))
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_performance_hierarchy(df, output_file=None):
    """
    Sunburst plot: Hierarchical Breakdown (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Score', 'Country', 'Organization'])
    org_scores = df_plot.groupby(['Country', 'Organization'])['Score'].mean().reset_index()
    fig = px.sunburst(org_scores, path=['Country', 'Organization'], values='Score',
                      title='Performance Hierarchy: Country > Org',
                      color_discrete_sequence=NEON_PALETTE)
    fig.update_layout(template='plotly_dark', font=dict(color='white'))
    if output_file:
        fig.write_html(output_file)
    fig.show()
def create_dashboard(df, output_file='boolq_dashboard.html'):
    """
    Combined dashboard (dark theme) with Markdown write-up inserted at the top.
    """
    df_plot = prepare_data(df)
    fig = make_subplots(rows=2, cols=2,
                        subplot_titles=('Score Over Time', 'Scaling Curve',
                                        'Bar by Org', 'Score Distribution by Country'),
                        specs=[
                            [{"secondary_y": False}, {"secondary_y": False}],
                            [{"secondary_y": False}, {"secondary_y": False}]
                        ])
    # Line plot
    df_line = df_plot.dropna(subset=['Release date', 'Score']).sort_values('Release date')
    line_fig = px.line(df_line, x='Release date', y='Score', color='Organization', markers=True,
                       color_discrete_sequence=NEON_PALETTE)
    for trace in line_fig.data:
        fig.add_trace(trace, row=1, col=1)
    # Scatter plot
    df_scatter = df_plot.dropna(subset=['Score', 'Training compute (FLOP)'])
    df_scatter = df_scatter[df_scatter['Training compute (FLOP)'] > 0]
    scatter_fig = px.scatter(df_scatter, x='Training compute (FLOP)', y='Score', log_x=True,
                             color='Shots', color_discrete_sequence=NEON_PALETTE)
    for trace in scatter_fig.data:
        fig.add_trace(trace, row=1, col=2)
    # Bar plot: Average Score by Org
    df_bar = df_plot.dropna(subset=['Score'])
    df_grouped_bar = df_bar.groupby('Organization')['Score'].mean().reset_index()
    bar_fig = px.bar(df_grouped_bar, x='Organization', y='Score',
                     color_discrete_sequence=NEON_PALETTE)
    for trace in bar_fig.data:
        fig.add_trace(trace, row=2, col=1)
    # Box plot: Score by Country
    df_box = df_plot.dropna(subset=['Score'])
    box_fig = px.box(df_box, x='Country', y='Score', points='outliers',
                     color_discrete_sequence=NEON_PALETTE)
    for trace in box_fig.data:
        fig.add_trace(trace, row=2, col=2)
    fig.update_layout(height=800, title_text="BoolQ Model Performance Dashboard (Dark Neon Theme)",
                      template='plotly_dark', font=dict(color='white'))
   
    # Generate HTML string
    html_content = fig.to_html(full_html=True, include_plotlyjs='cdn')
   
    # Markdown write-up as HTML (dark-themed)
    writeup_html = '''
    <div style="padding: 20px; background-color: #1f1f1f; color: #ffffff; border-bottom: 2px solid #333333; font-family: Arial, sans-serif;">
        <h2 style="color: #00FFFF; margin-bottom: 15px;">BoolQ</h2>
        <p style="line-height: 1.6; font-size: 14px;">
            BoolQ is a question answering dataset focused on yes/no questions derived from Google searches and Wikipedia passages.
            It tests models' ability to comprehend passages and determine binary answers, emphasizing reading comprehension and factual inference without requiring external knowledge.
            Questions are crowdsourced to ensure natural phrasing, making it a robust benchmark for extractive QA in realistic scenarios.
        </p>
        <p style="line-height: 1.6; font-size: 14px; margin-top: 10px;">
            The <strong>Score</strong> reflects accuracy on this task, serving as an indicator of models' natural language understanding and decision-making precision.
            BoolQ has become a standard in evaluating progress in passage-based QA, highlighting improvements in contextual reasoning.
        </p>
    </div>
    '''
   
    # Improved insertion: after scripts, before plot div
    body_start = html_content.find('<body>')
    if body_start != -1:
        plot_div_class_pos = html_content.find('class="plotly-graph-div"', body_start)
        if plot_div_class_pos != -1:
            div_start = html_content.rfind('<div', body_start, plot_div_class_pos)
            if div_start != -1:
                full_html = html_content[:div_start] + writeup_html + html_content[div_start:]
            else:
                full_html = html_content
        else:
            full_html = html_content
    else:
        full_html = html_content
   
    # Write to file
    if output_file:
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(full_html)
   
    fig.show()
    print(f"Dashboard saved to {output_file} with BoolQ write-up rendered at the top.")
# Preview (as requested)
plot_score_over_time(df)
create_dashboard(df)

Dashboard saved to boolq_dashboard.html with BoolQ write-up rendered at the top.


In [42]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import io
# Neon color palette (bright colors for contrast on dark bg)
NEON_PALETTE = ['#FF00FF', '#00FFFF', '#FFFF00', '#FF0080', '#00FF80', '#8000FF', '#FF8000', '#00FF00']
def prepare_data(df):
    """
    Prepare the DataFrame for plotting.
    """
    df = df.copy()
    if 'Release date' in df.columns:
        df['Release date'] = pd.to_datetime(df['Release date'], errors='coerce')
    # Handle Average: fill NaN with 0
    numeric_cols = ['Average']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            df[col] = df[col].fillna(0)
    # Handle Training compute (FLOP): convert to numeric, leave NaN as NaN (to avoid log issues)
    if 'Training compute (FLOP)' in df.columns:
        df['Training compute (FLOP)'] = pd.to_numeric(df['Training compute (FLOP)'], errors='coerce')
    if 'Model version' in df.columns and 'Name' in df.columns:
        df['Model version'] = df['Model version'].fillna(df['Name'])
    return df
def plot_average_over_time(df, output_file=None):
    """
    Line plot: Model Average Evolution by Release Date (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Release date', 'Average'])
    df_plot = df_plot.sort_values('Release date')
    fig = px.line(df_plot, x='Release date', y='Average', color='Organization',
                  title='BBH Average Score Trends Over Time', markers=True,
                  color_discrete_sequence=NEON_PALETTE)
    fig.update_xaxes(type='date')
    fig.update_layout(template='plotly_dark', font=dict(color='white')) # Dark theme + white text
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_bar_by_org(df, output_file=None):
    """
    Bar plot: Average Score by Organization (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Average'])
    df_grouped = df_plot.groupby('Organization')['Average'].mean().reset_index()
    fig = px.bar(df_grouped, x='Organization', y='Average',
                 title='Average BBH Score by Organization',
                 color_discrete_sequence=NEON_PALETTE)
    fig.update_layout(template='plotly_dark', font=dict(color='white')) # Dark theme + white text
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_compute_growth(df, output_file=None):
    """
    Area plot: Cumulative Compute vs. Release Date (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Release date', 'Training compute (FLOP)'])
    df_plot = df_plot[df_plot['Training compute (FLOP)'] > 0]
    df_plot = df_plot.sort_values('Release date')
    fig = px.area(df_plot, x='Release date', y='Training compute (FLOP)', color='Country',
                  title='Model Compute Growth by Country',
                  color_discrete_sequence=NEON_PALETTE)
    fig.update_xaxes(type='date')
    fig.update_layout(template='plotly_dark', font=dict(color='white'))
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_average_distribution(df, output_file=None):
    """
    Box plot: Average Score Distribution by Country (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Average'])
    fig = px.box(df_plot, x='Country', y='Average', points='outliers',
                 title='Average Score Distributions by Country',
                 color_discrete_sequence=NEON_PALETTE)
    fig.update_layout(template='plotly_dark', font=dict(color='white'))
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_scaling_curve(df, output_file=None):
    """
    Scatter plot: Average vs. Training Compute (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Average', 'Training compute (FLOP)'])
    df_plot = df_plot[df_plot['Training compute (FLOP)'] > 0]
    fig = px.scatter(df_plot, x='Training compute (FLOP)', y='Average', color='Shots',
                     log_x=True, trendline='ols',
                     title='Average vs. Compute (Scaling Curve)',
                     color_discrete_sequence=NEON_PALETTE)
    fig.update_layout(template='plotly_dark', font=dict(color='white'))
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_average_bubbles(df, output_file=None):
    """
    Bubble chart: Average Performance vs. Release Date (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Release date', 'Average', 'Training compute (FLOP)'])
    df_plot = df_plot[df_plot['Training compute (FLOP)'] > 0]
    df_plot = df_plot.sort_values('Release date')
    fig = px.scatter(df_plot, x='Release date', y='Average', size='Training compute (FLOP)',
                     color='Organization', hover_name='Name',
                     title='BBH Average Scores Over Time (Bubble = Compute)',
                     color_discrete_sequence=NEON_PALETTE)
    fig.update_xaxes(type='date')
    fig.update_layout(template='plotly_dark', font=dict(color='white'))
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_performance_hierarchy(df, output_file=None):
    """
    Sunburst plot: Hierarchical Breakdown (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Average', 'Country', 'Organization'])
    org_scores = df_plot.groupby(['Country', 'Organization'])['Average'].mean().reset_index()
    fig = px.sunburst(org_scores, path=['Country', 'Organization'], values='Average',
                      title='Performance Hierarchy: Country > Org',
                      color_discrete_sequence=NEON_PALETTE)
    fig.update_layout(template='plotly_dark', font=dict(color='white'))
    if output_file:
        fig.write_html(output_file)
    fig.show()
def create_dashboard(df, output_file='bbh_dashboard.html'):
    """
    Combined dashboard (dark theme) with Markdown write-up inserted at the top.
    """
    df_plot = prepare_data(df)
    fig = make_subplots(rows=2, cols=2,
                        subplot_titles=('Average Over Time', 'Scaling Curve',
                                        'Bar by Org', 'Average Distribution by Country'),
                        specs=[
                            [{"secondary_y": False}, {"secondary_y": False}],
                            [{"secondary_y": False}, {"secondary_y": False}]
                        ])
    # Line plot
    df_line = df_plot.dropna(subset=['Release date', 'Average']).sort_values('Release date')
    line_fig = px.line(df_line, x='Release date', y='Average', color='Organization', markers=True,
                       color_discrete_sequence=NEON_PALETTE)
    for trace in line_fig.data:
        fig.add_trace(trace, row=1, col=1)
    # Scatter plot
    df_scatter = df_plot.dropna(subset=['Average', 'Training compute (FLOP)'])
    df_scatter = df_scatter[df_scatter['Training compute (FLOP)'] > 0]
    scatter_fig = px.scatter(df_scatter, x='Training compute (FLOP)', y='Average', log_x=True,
                             color='Shots', color_discrete_sequence=NEON_PALETTE)
    for trace in scatter_fig.data:
        fig.add_trace(trace, row=1, col=2)
    # Bar plot: Average by Org
    df_bar = df_plot.dropna(subset=['Average'])
    df_grouped_bar = df_bar.groupby('Organization')['Average'].mean().reset_index()
    bar_fig = px.bar(df_grouped_bar, x='Organization', y='Average',
                     color_discrete_sequence=NEON_PALETTE)
    for trace in bar_fig.data:
        fig.add_trace(trace, row=2, col=1)
    # Box plot: Average by Country
    df_box = df_plot.dropna(subset=['Average'])
    box_fig = px.box(df_box, x='Country', y='Average', points='outliers',
                     color_discrete_sequence=NEON_PALETTE)
    for trace in box_fig.data:
        fig.add_trace(trace, row=2, col=2)
    fig.update_layout(height=800, title_text="BBH Model Performance Dashboard (Dark Neon Theme)",
                      template='plotly_dark', font=dict(color='white'))
   
    # Generate HTML string
    html_content = fig.to_html(full_html=True, include_plotlyjs='cdn')
   
    # Markdown write-up as HTML (dark-themed)
    writeup_html = '''
    <div style="padding: 20px; background-color: #1f1f1f; color: #ffffff; border-bottom: 2px solid #333333; font-family: Arial, sans-serif;">
        <h2 style="color: #00FFFF; margin-bottom: 15px;">BIG-Bench Hard (BBH)</h2>
        <p style="line-height: 1.6; font-size: 14px;">
            BIG-Bench Hard (BBH) is a challenging subset of the BIG-Bench benchmark, curated to focus on tasks where even the strongest language models at the time achieved low performance (&lt; 50%).
            It includes 23 diverse tasks spanning navigation, logical deduction, multi-step arithmetic, and causal reasoning, designed to probe the limits of model capabilities in complex, human-like reasoning.
            BBH emphasizes tasks requiring few-shot learning and generalization beyond training data patterns.
        </p>
        <p style="line-height: 1.6; font-size: 14px; margin-top: 10px;">
            The <strong>Average</strong> score aggregates performance across these tasks, providing a robust measure of frontier model progress. BBH remains a critical evaluation for assessing advancements in reasoning and problem-solving.
        </p>
    </div>
    '''
   
    # Improved insertion: after scripts, before plot div
    body_start = html_content.find('<body>')
    if body_start != -1:
        plot_div_class_pos = html_content.find('class="plotly-graph-div"', body_start)
        if plot_div_class_pos != -1:
            div_start = html_content.rfind('<div', body_start, plot_div_class_pos)
            if div_start != -1:
                full_html = html_content[:div_start] + writeup_html + html_content[div_start:]
            else:
                full_html = html_content
        else:
            full_html = html_content
    else:
        full_html = html_content
   
    # Write to file
    if output_file:
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(full_html)
   
    fig.show()
    print(f"Dashboard saved to {output_file} with BBH write-up rendered at the top.")
# Preview (as requested)
plot_average_over_time(df)
create_dashboard(df)

Dashboard saved to bbh_dashboard.html with BBH write-up rendered at the top.


In [39]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import io
# Neon color palette (bright colors for contrast on dark bg)
NEON_PALETTE = ['#FF00FF', '#00FFFF', '#FFFF00', '#FF0080', '#00FF80', '#8000FF', '#FF8000', '#00FF00']
def prepare_data(df):
    """
    Prepare the DataFrame for plotting.
    """
    df = df.copy()
    if 'Release date' in df.columns:
        df['Release date'] = pd.to_datetime(df['Release date'], errors='coerce')
    # Handle Average progress and sub-progresses: fill NaN with 0
    numeric_cols = ['Average progress', 'BabyAI progress', 'Crafter progress', 'TextWorld progress',
                    'BabalsAI progress', 'MiniHack progress', 'NetHack progress']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            df[col] = df[col].fillna(0)
    # Handle Training compute (FLOP): convert to numeric, leave NaN as NaN (to avoid log issues)
    if 'Training compute (FLOP)' in df.columns:
        df['Training compute (FLOP)'] = pd.to_numeric(df['Training compute (FLOP)'], errors='coerce')
    if 'Model version' in df.columns and 'Name' in df.columns:
        df['Model version'] = df['Model version'].fillna(df['Name'])
    return df
def plot_average_progress_over_time(df, output_file=None):
    """
    Line plot: Model Average Progress Evolution by Release Date (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Release date', 'Average progress'])
    df_plot = df_plot.sort_values('Release date')
    fig = px.line(df_plot, x='Release date', y='Average progress', color='Organization',
                  title='BALROG Average Progress Trends Over Time', markers=True,
                  color_discrete_sequence=NEON_PALETTE)
    fig.update_xaxes(type='date')
    fig.update_layout(template='plotly_dark', font=dict(color='white')) # Dark theme + white text
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_bar_by_org(df, output_file=None):
    """
    Bar plot: Average Progress by Organization (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Average progress'])
    df_grouped = df_plot.groupby('Organization')['Average progress'].mean().reset_index()
    fig = px.bar(df_grouped, x='Organization', y='Average progress',
                 title='Average BALROG Progress by Organization',
                 color_discrete_sequence=NEON_PALETTE)
    fig.update_layout(template='plotly_dark', font=dict(color='white')) # Dark theme + white text
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_compute_growth(df, output_file=None):
    """
    Area plot: Cumulative Compute vs. Release Date (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Release date', 'Training compute (FLOP)'])
    df_plot = df_plot[df_plot['Training compute (FLOP)'] > 0]
    df_plot = df_plot.sort_values('Release date')
    fig = px.area(df_plot, x='Release date', y='Training compute (FLOP)', color='Country',
                  title='Model Compute Growth by Country',
                  color_discrete_sequence=NEON_PALETTE)
    fig.update_xaxes(type='date')
    fig.update_layout(template='plotly_dark', font=dict(color='white'))
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_average_progress_distribution(df, output_file=None):
    """
    Box plot: Average Progress Distribution by Country (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Average progress'])
    fig = px.box(df_plot, x='Country', y='Average progress', points='outliers',
                 title='Average Progress Distributions by Country',
                 color_discrete_sequence=NEON_PALETTE)
    fig.update_layout(template='plotly_dark', font=dict(color='white'))
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_scaling_curve(df, output_file=None):
    """
    Scatter plot: Average Progress vs. Training Compute (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Average progress', 'Training compute (FLOP)'])
    df_plot = df_plot[df_plot['Training compute (FLOP)'] > 0]
    fig = px.scatter(df_plot, x='Training compute (FLOP)', y='Average progress', color='Organization',
                     log_x=True, trendline='ols',
                     title='Average Progress vs. Compute (Scaling Curve)',
                     color_discrete_sequence=NEON_PALETTE)
    fig.update_layout(template='plotly_dark', font=dict(color='white'))
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_average_progress_bubbles(df, output_file=None):
    """
    Bubble chart: Average Progress Performance vs. Release Date (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Release date', 'Average progress', 'Training compute (FLOP)'])
    df_plot = df_plot[df_plot['Training compute (FLOP)'] > 0]
    df_plot = df_plot.sort_values('Release date')
    fig = px.scatter(df_plot, x='Release date', y='Average progress', size='Training compute (FLOP)',
                     color='Organization', hover_name='Name',
                     title='BALROG Average Progress Over Time (Bubble = Compute)',
                     color_discrete_sequence=NEON_PALETTE)
    fig.update_xaxes(type='date')
    fig.update_layout(template='plotly_dark', font=dict(color='white'))
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_correlation_heatmap(df, output_file=None):
    """
    Heatmap: Correlation Matrix (dark theme with neon-inspired scale).
    """
    df_plot = prepare_data(df)
    corr_cols = ['Average progress', 'BabyAI progress', 'Crafter progress', 'TextWorld progress']
    df_plot_corr = df_plot.dropna(subset=corr_cols)
    corr_df = df_plot_corr[corr_cols].corr()
    fig = go.Figure(data=go.Heatmap(z=corr_df.values, x=corr_df.columns, y=corr_df.index,
                                    colorscale='Plasma'))
    fig.update_layout(template='plotly_dark', title='BALROG Metrics Correlation Heatmap', font=dict(color='white'))
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_performance_hierarchy(df, output_file=None):
    """
    Sunburst plot: Hierarchical Breakdown (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Average progress', 'Country', 'Organization'])
    org_scores = df_plot.groupby(['Country', 'Organization'])['Average progress'].mean().reset_index()
    fig = px.sunburst(org_scores, path=['Country', 'Organization'], values='Average progress',
                      title='Performance Hierarchy: Country > Org',
                      color_discrete_sequence=NEON_PALETTE)
    fig.update_layout(template='plotly_dark', font=dict(color='white'))
    if output_file:
        fig.write_html(output_file)
    fig.show()
def create_dashboard(df, output_file='balrog_dashboard.html'):
    """
    Combined dashboard (dark theme) with Markdown write-up inserted at the top.
    """
    df_plot = prepare_data(df)
    fig = make_subplots(rows=2, cols=2,
                        subplot_titles=('Average Progress Over Time', 'Scaling Curve',
                                        'Bar by Org', 'Correlation Heatmap'),
                        specs=[
                            [{"secondary_y": False}, {"secondary_y": False}],
                            [{"secondary_y": False}, {"type": "heatmap"}]
                        ])
    # Line plot
    df_line = df_plot.dropna(subset=['Release date', 'Average progress']).sort_values('Release date')
    line_fig = px.line(df_line, x='Release date', y='Average progress', color='Organization', markers=True,
                       color_discrete_sequence=NEON_PALETTE)
    for trace in line_fig.data:
        fig.add_trace(trace, row=1, col=1)
    # Scatter plot
    df_scatter = df_plot.dropna(subset=['Average progress', 'Training compute (FLOP)'])
    df_scatter = df_scatter[df_scatter['Training compute (FLOP)'] > 0]
    scatter_fig = px.scatter(df_scatter, x='Training compute (FLOP)', y='Average progress', log_x=True,
                             color_discrete_sequence=NEON_PALETTE)
    for trace in scatter_fig.data:
        fig.add_trace(trace, row=1, col=2)
    # Bar plot: Average Progress by Org
    df_bar = df_plot.dropna(subset=['Average progress'])
    df_grouped_bar = df_bar.groupby('Organization')['Average progress'].mean().reset_index()
    bar_fig = px.bar(df_grouped_bar, x='Organization', y='Average progress',
                     color_discrete_sequence=NEON_PALETTE)
    for trace in bar_fig.data:
        fig.add_trace(trace, row=2, col=1)
    # Heatmap
    corr_cols = ['Average progress', 'BabyAI progress', 'Crafter progress', 'TextWorld progress']
    df_heatmap = df_plot.dropna(subset=corr_cols)
    corr_df = df_heatmap[corr_cols].corr()
    heatmap = go.Heatmap(z=corr_df.values, x=corr_df.columns, y=corr_df.index, colorscale='Plasma')
    fig.add_trace(heatmap, row=2, col=2)
    fig.update_layout(height=800, title_text="BALROG Model Performance Dashboard (Dark Neon Theme)",
                      template='plotly_dark', font=dict(color='white'))
   
    # Generate HTML string
    html_content = fig.to_html(full_html=True, include_plotlyjs='cdn')
   
    # Markdown write-up as HTML (dark-themed)
    writeup_html = '''
    <div style="padding: 20px; background-color: #1f1f1f; color: #ffffff; border-bottom: 2px solid #333333; font-family: Arial, sans-serif;">
        <h2 style="color: #00FFFF; margin-bottom: 15px;">BALROG Benchmark</h2>
        <p style="line-height: 1.6; font-size: 14px;">
            BALROG is a novel benchmark designed to evaluate the agentic capabilities of large language models (LLMs) and vision-language models (VLMs) on long-horizon interactive tasks.
            It leverages challenging reinforcement learning environments including BabyAI, Crafter, TextWorld, BabyalsAI, MiniHack, and NetHack to assess reasoning, planning, and execution in diverse game-like scenarios.
            Models are prompted to act as agents, generating actions to maximize progress toward goals, simulating real-world agentic behavior.
        </p>
        <p style="line-height: 1.6; font-size: 14px; margin-top: 10px;">
            The <strong>Average progress</strong> aggregates performance across environments, with sub-metrics providing granular insights into specific domains. This benchmark highlights gaps in current models' ability to handle complex, multi-step reasoning under partial observability.
        </p>
    </div>
    '''
   
    # Improved insertion: after scripts, before plot div
    body_start = html_content.find('<body>')
    if body_start != -1:
        plot_div_class_pos = html_content.find('class="plotly-graph-div"', body_start)
        if plot_div_class_pos != -1:
            div_start = html_content.rfind('<div', body_start, plot_div_class_pos)
            if div_start != -1:
                full_html = html_content[:div_start] + writeup_html + html_content[div_start:]
            else:
                full_html = html_content
        else:
            full_html = html_content
    else:
        full_html = html_content
   
    # Write to file
    if output_file:
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(full_html)
   
    fig.show()
    print(f"Dashboard saved to {output_file} with BALROG write-up rendered at the top.")
# Preview (as requested)
plot_average_progress_over_time(df)
create_dashboard(df)

Dashboard saved to balrog_dashboard.html with BALROG write-up rendered at the top.


In [36]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import io
# Neon color palette (bright colors for contrast on dark bg)
NEON_PALETTE = ['#FF00FF', '#00FFFF', '#FFFF00', '#FF0080', '#00FF80', '#8000FF', '#FF8000', '#00FF00']
def prepare_data(df):
    """
    Prepare the DataFrame for plotting.
    """
    df = df.copy()
    if 'Release date' in df.columns:
        df['Release date'] = pd.to_datetime(df['Release date'], errors='coerce')
    # Handle Challenge score, Easy score: fill NaN with 0
    numeric_cols = ['Challenge score', 'Easy score']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            df[col] = df[col].fillna(0)
    # Handle Training compute (FLOP): convert to numeric, leave NaN as NaN (to avoid log issues)
    if 'Training compute (FLOP)' in df.columns:
        df['Training compute (FLOP)'] = pd.to_numeric(df['Training compute (FLOP)'], errors='coerce')
    if 'Model version' in df.columns and 'Name' in df.columns:
        df['Model version'] = df['Model version'].fillna(df['Name'])
    return df
def plot_challenge_score_over_time(df, output_file=None):
    """
    Line plot: Model Challenge Score Evolution by Release Date (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Release date', 'Challenge score'])
    df_plot = df_plot.sort_values('Release date')
    fig = px.line(df_plot, x='Release date', y='Challenge score', color='Organization',
                  title='ARC AI2 Challenge Score Trends Over Time', markers=True,
                  color_discrete_sequence=NEON_PALETTE)
    fig.update_xaxes(type='date')
    fig.update_layout(template='plotly_dark', font=dict(color='white')) # Dark theme + white text
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_bar_by_org(df, output_file=None):
    """
    Bar plot: Average Challenge Score by Organization (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Challenge score'])
    df_grouped = df_plot.groupby('Organization')['Challenge score'].mean().reset_index()
    fig = px.bar(df_grouped, x='Organization', y='Challenge score',
                 title='Average ARC AI2 Challenge Score by Organization',
                 color_discrete_sequence=NEON_PALETTE)
    fig.update_layout(template='plotly_dark', font=dict(color='white')) # Dark theme + white text
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_compute_growth(df, output_file=None):
    """
    Area plot: Cumulative Compute vs. Release Date (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Release date', 'Training compute (FLOP)'])
    df_plot = df_plot[df_plot['Training compute (FLOP)'] > 0]
    df_plot = df_plot.sort_values('Release date')
    fig = px.area(df_plot, x='Release date', y='Training compute (FLOP)', color='Country',
                  title='Model Compute Growth by Country',
                  color_discrete_sequence=NEON_PALETTE)
    fig.update_xaxes(type='date')
    fig.update_layout(template='plotly_dark', font=dict(color='white'))
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_challenge_score_distribution(df, output_file=None):
    """
    Box plot: Challenge Score Distribution by Country (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Challenge score'])
    fig = px.box(df_plot, x='Country', y='Challenge score', points='outliers',
                 title='Challenge Score Distributions by Country',
                 color_discrete_sequence=NEON_PALETTE)
    fig.update_layout(template='plotly_dark', font=dict(color='white'))
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_scaling_curve(df, output_file=None):
    """
    Scatter plot: Challenge Score vs. Training Compute (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Challenge score', 'Training compute (FLOP)'])
    df_plot = df_plot[df_plot['Training compute (FLOP)'] > 0]
    fig = px.scatter(df_plot, x='Training compute (FLOP)', y='Challenge score', color='Shots',
                     log_x=True, trendline='ols',
                     title='Challenge Score vs. Compute (Scaling Curve)',
                     color_discrete_sequence=NEON_PALETTE)
    fig.update_layout(template='plotly_dark', font=dict(color='white'))
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_challenge_bubbles(df, output_file=None):
    """
    Bubble chart: Challenge Score Performance vs. Release Date (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Release date', 'Challenge score', 'Training compute (FLOP)'])
    df_plot = df_plot[df_plot['Training compute (FLOP)'] > 0]
    df_plot = df_plot.sort_values('Release date')
    fig = px.scatter(df_plot, x='Release date', y='Challenge score', size='Training compute (FLOP)',
                     color='Organization', hover_name='Name',
                     title='ARC AI2 Challenge Scores Over Time (Bubble = Compute)',
                     color_discrete_sequence=NEON_PALETTE)
    fig.update_xaxes(type='date')
    fig.update_layout(template='plotly_dark', font=dict(color='white'))
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_correlation_heatmap(df, output_file=None):
    """
    Heatmap: Correlation Matrix (dark theme with neon-inspired scale).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Easy score', 'Challenge score'])
    corr_cols = ['Easy score', 'Challenge score']
    corr_df = df_plot[corr_cols].corr()
    fig = go.Figure(data=go.Heatmap(z=corr_df.values, x=corr_df.columns, y=corr_df.index,
                                    colorscale='Plasma'))
    fig.update_layout(template='plotly_dark', title='ARC AI2 Metrics Correlation Heatmap', font=dict(color='white'))
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_performance_hierarchy(df, output_file=None):
    """
    Sunburst plot: Hierarchical Breakdown (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Challenge score', 'Country', 'Organization'])
    org_scores = df_plot.groupby(['Country', 'Organization'])['Challenge score'].mean().reset_index()
    fig = px.sunburst(org_scores, path=['Country', 'Organization'], values='Challenge score',
                      title='Performance Hierarchy: Country > Org',
                      color_discrete_sequence=NEON_PALETTE)
    fig.update_layout(template='plotly_dark', font=dict(color='white'))
    if output_file:
        fig.write_html(output_file)
    fig.show()
def create_dashboard(df, output_file='arc_ai2_dashboard.html'):
    """
    Combined dashboard (dark theme) with Markdown write-up inserted at the top.
    """
    df_plot = prepare_data(df)
    fig = make_subplots(rows=2, cols=2,
                        subplot_titles=('Challenge Score Over Time', 'Scaling Curve',
                                        'Bar by Org', 'Correlation Heatmap'),
                        specs=[
                            [{"secondary_y": False}, {"secondary_y": False}],
                            [{"secondary_y": False}, {"type": "heatmap"}]
                        ])
    # Line plot
    df_line = df_plot.dropna(subset=['Release date', 'Challenge score']).sort_values('Release date')
    line_fig = px.line(df_line, x='Release date', y='Challenge score', color='Organization', markers=True,
                       color_discrete_sequence=NEON_PALETTE)
    for trace in line_fig.data:
        fig.add_trace(trace, row=1, col=1)
    # Scatter plot
    df_scatter = df_plot.dropna(subset=['Challenge score', 'Training compute (FLOP)'])
    df_scatter = df_scatter[df_scatter['Training compute (FLOP)'] > 0]
    scatter_fig = px.scatter(df_scatter, x='Training compute (FLOP)', y='Challenge score', log_x=True,
                             color_discrete_sequence=NEON_PALETTE)
    for trace in scatter_fig.data:
        fig.add_trace(trace, row=1, col=2)
    # Bar plot: Average Challenge Score by Org
    df_bar = df_plot.dropna(subset=['Challenge score'])
    df_grouped_bar = df_bar.groupby('Organization')['Challenge score'].mean().reset_index()
    bar_fig = px.bar(df_grouped_bar, x='Organization', y='Challenge score',
                     color_discrete_sequence=NEON_PALETTE)
    for trace in bar_fig.data:
        fig.add_trace(trace, row=2, col=1)
    # Heatmap
    df_heatmap = df_plot.dropna(subset=['Easy score', 'Challenge score'])
    corr_cols = ['Easy score', 'Challenge score']
    corr_df = df_heatmap[corr_cols].corr()
    heatmap = go.Heatmap(z=corr_df.values, x=corr_df.columns, y=corr_df.index, colorscale='Plasma')
    fig.add_trace(heatmap, row=2, col=2)
    fig.update_layout(height=800, title_text="ARC AI2 Model Performance Dashboard (Dark Neon Theme)",
                      template='plotly_dark', font=dict(color='white'))
   
    # Generate HTML string
    html_content = fig.to_html(full_html=True, include_plotlyjs='cdn')
   
    # Markdown write-up as HTML (dark-themed)
    writeup_html = '''
    <div style="padding: 20px; background-color: #1f1f1f; color: #ffffff; border-bottom: 2px solid #333333; font-family: Arial, sans-serif;">
        <h2 style="color: #00FFFF; margin-bottom: 15px;">ARC Challenge (AI2)</h2>
        <p style="line-height: 1.6; font-size: 14px;">
            The ARC Challenge, developed by the Allen Institute for AI (AI2), is an advanced benchmark for assessing abstraction and reasoning in AI systems.
            It builds on the original ARC dataset with a focus on core knowledge priors like objectness, goal-directedness, and basic physics, using grid-based puzzles that require few-shot generalization.
            Tasks are divided into <strong>Easy score</strong> for simpler demonstrations and <strong>Challenge score</strong> for the harder, unseen test cases that truly probe reasoning abilities.
        </p>
        <p style="line-height: 1.6; font-size: 14px; margin-top: 10px;">
            This benchmark emphasizes efficiency and novel problem-solving over scale, revealing limitations in current LLMs and guiding research toward more human-like intelligence.
        </p>
    </div>
    '''
   
    # Improved insertion: after scripts, before plot div
    body_start = html_content.find('<body>')
    if body_start != -1:
        plot_div_class_pos = html_content.find('class="plotly-graph-div"', body_start)
        if plot_div_class_pos != -1:
            div_start = html_content.rfind('<div', body_start, plot_div_class_pos)
            if div_start != -1:
                full_html = html_content[:div_start] + writeup_html + html_content[div_start:]
            else:
                full_html = html_content
        else:
            full_html = html_content
    else:
        full_html = html_content
   
    # Write to file
    if output_file:
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(full_html)
   
    fig.show()
    print(f"Dashboard saved to {output_file} with ARC AI2 write-up rendered at the top.")
# Preview (as requested)
plot_challenge_score_over_time(df)
create_dashboard(df)

Dashboard saved to arc_ai2_dashboard.html with ARC AI2 write-up rendered at the top.


In [33]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import io
# Neon color palette (bright colors for contrast on dark bg)
NEON_PALETTE = ['#FF00FF', '#00FFFF', '#FFFF00', '#FF0080', '#00FF80', '#8000FF', '#FF8000', '#00FF00']
def prepare_data(df):
    """
    Prepare the DataFrame for plotting.
    """
    df = df.copy()
    if 'Release date' in df.columns:
        df['Release date'] = pd.to_datetime(df['Release date'], errors='coerce')
    # Handle Score: fill NaN with 0
    numeric_cols = ['Score']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            df[col] = df[col].fillna(0)
    # Handle Training compute (FLOP): convert to numeric, leave NaN as NaN (to avoid log issues)
    if 'Training compute (FLOP)' in df.columns:
        df['Training compute (FLOP)'] = pd.to_numeric(df['Training compute (FLOP)'], errors='coerce')
    if 'Model version' in df.columns and 'Name' in df.columns:
        df['Model version'] = df['Model version'].fillna(df['Name'])
    return df
def plot_score_over_time(df, output_file=None):
    """
    Line plot: Model Score Evolution by Release Date (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Release date', 'Score'])
    df_plot = df_plot.sort_values('Release date')
    fig = px.line(df_plot, x='Release date', y='Score', color='Organization',
                  title='ARC-AGI Score Trends Over Time', markers=True,
                  color_discrete_sequence=NEON_PALETTE)
    fig.update_xaxes(type='date')
    fig.update_layout(template='plotly_dark', font=dict(color='white')) # Dark theme + white text
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_bar_by_org(df, output_file=None):
    """
    Bar plot: Average Score by Organization (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Score'])
    df_grouped = df_plot.groupby('Organization')['Score'].mean().reset_index()
    fig = px.bar(df_grouped, x='Organization', y='Score',
                 title='Average ARC-AGI Score by Organization',
                 color_discrete_sequence=NEON_PALETTE)
    fig.update_layout(template='plotly_dark', font=dict(color='white')) # Dark theme + white text
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_compute_growth(df, output_file=None):
    """
    Area plot: Cumulative Compute vs. Release Date (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Release date', 'Training compute (FLOP)'])
    df_plot = df_plot[df_plot['Training compute (FLOP)'] > 0]
    df_plot = df_plot.sort_values('Release date')
    fig = px.area(df_plot, x='Release date', y='Training compute (FLOP)', color='Country',
                  title='Model Compute Growth by Country',
                  color_discrete_sequence=NEON_PALETTE)
    fig.update_xaxes(type='date')
    fig.update_layout(template='plotly_dark', font=dict(color='white'))
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_score_distribution(df, output_file=None):
    """
    Box plot: Score Distribution by Country (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Score'])
    fig = px.box(df_plot, x='Country', y='Score', points='outliers',
                 title='Score Distributions by Country',
                 color_discrete_sequence=NEON_PALETTE)
    fig.update_layout(template='plotly_dark', font=dict(color='white'))
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_scaling_curve(df, output_file=None):
    """
    Scatter plot: Score vs. Training Compute (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Score', 'Training compute (FLOP)'])
    df_plot = df_plot[df_plot['Training compute (FLOP)'] > 0]
    fig = px.scatter(df_plot, x='Training compute (FLOP)', y='Score', color='Organization',
                     log_x=True, trendline='ols',
                     title='Score vs. Compute (Scaling Curve)',
                     color_discrete_sequence=NEON_PALETTE)
    fig.update_layout(template='plotly_dark', font=dict(color='white'))
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_score_bubbles(df, output_file=None):
    """
    Bubble chart: Score Performance vs. Release Date (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Release date', 'Score', 'Training compute (FLOP)'])
    df_plot = df_plot[df_plot['Training compute (FLOP)'] > 0]
    df_plot = df_plot.sort_values('Release date')
    fig = px.scatter(df_plot, x='Release date', y='Score', size='Training compute (FLOP)',
                     color='Organization', hover_name='Name',
                     title='ARC-AGI Scores Over Time (Bubble = Compute)',
                     color_discrete_sequence=NEON_PALETTE)
    fig.update_xaxes(type='date')
    fig.update_layout(template='plotly_dark', font=dict(color='white'))
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_performance_hierarchy(df, output_file=None):
    """
    Sunburst plot: Hierarchical Breakdown (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Score', 'Country', 'Organization'])
    org_scores = df_plot.groupby(['Country', 'Organization'])['Score'].mean().reset_index()
    fig = px.sunburst(org_scores, path=['Country', 'Organization'], values='Score',
                      title='Performance Hierarchy: Country > Org',
                      color_discrete_sequence=NEON_PALETTE)
    fig.update_layout(template='plotly_dark', font=dict(color='white'))
    if output_file:
        fig.write_html(output_file)
    fig.show()
def create_dashboard(df, output_file='arc_agi_dashboard.html'):
    """
    Combined dashboard (dark theme) with Markdown write-up inserted at the top.
    """
    df_plot = prepare_data(df)
    fig = make_subplots(rows=2, cols=2,
                        subplot_titles=('Score Over Time', 'Scaling Curve',
                                        'Bar by Org', 'Score Distribution by Country'),
                        specs=[
                            [{"secondary_y": False}, {"secondary_y": False}],
                            [{"secondary_y": False}, {"secondary_y": False}]
                        ])
    # Line plot
    df_line = df_plot.dropna(subset=['Release date', 'Score']).sort_values('Release date')
    line_fig = px.line(df_line, x='Release date', y='Score', color='Organization', markers=True,
                       color_discrete_sequence=NEON_PALETTE)
    for trace in line_fig.data:
        fig.add_trace(trace, row=1, col=1)
    # Scatter plot
    df_scatter = df_plot.dropna(subset=['Score', 'Training compute (FLOP)'])
    df_scatter = df_scatter[df_scatter['Training compute (FLOP)'] > 0]
    scatter_fig = px.scatter(df_scatter, x='Training compute (FLOP)', y='Score', log_x=True,
                             color='Organization', color_discrete_sequence=NEON_PALETTE)
    for trace in scatter_fig.data:
        fig.add_trace(trace, row=1, col=2)
    # Bar plot: Average Score by Org
    df_bar = df_plot.dropna(subset=['Score'])
    df_grouped_bar = df_bar.groupby('Organization')['Score'].mean().reset_index()
    bar_fig = px.bar(df_grouped_bar, x='Organization', y='Score',
                     color_discrete_sequence=NEON_PALETTE)
    for trace in bar_fig.data:
        fig.add_trace(trace, row=2, col=1)
    # Box plot: Score by Country
    df_box = df_plot.dropna(subset=['Score'])
    box_fig = px.box(df_box, x='Country', y='Score', points='outliers',
                     color_discrete_sequence=NEON_PALETTE)
    for trace in box_fig.data:
        fig.add_trace(trace, row=2, col=2)
    fig.update_layout(height=800, title_text="ARC-AGI Model Performance Dashboard (Dark Neon Theme)",
                      template='plotly_dark', font=dict(color='white'))
   
    # Generate HTML string
    html_content = fig.to_html(full_html=True, include_plotlyjs='cdn')
   
    # Markdown write-up as HTML (dark-themed)
    writeup_html = '''
    <div style="padding: 20px; background-color: #1f1f1f; color: #ffffff; border-bottom: 2px solid #333333; font-family: Arial, sans-serif;">
        <h2 style="color: #00FFFF; margin-bottom: 15px;">Abstraction and Reasoning Corpus (ARC-AGI)</h2>
        <p style="line-height: 1.6; font-size: 14px;">
            The ARC-AGI benchmark, developed by François Chollet, is a test of core intelligence focusing on abstraction and reasoning rather than memorization.
            It consists of grid-based puzzles where models must infer rules from few examples and apply them to novel test cases, emphasizing generalization and few-shot learning.
            Unlike traditional benchmarks, ARC-AGI uses procedurally generated tasks to avoid data leakage and measure true problem-solving capabilities.
        </p>
        <p style="line-height: 1.6; font-size: 14px; margin-top: 10px;">
            The <strong>Score</strong> represents the percentage of tasks solved correctly, serving as a key indicator for progress toward artificial general intelligence (AGI).
            Despite advances in scaling, top models still achieve low scores, underscoring the challenge of human-like reasoning.
        </p>
    </div>
    '''
   
    # Improved insertion: after scripts, before plot div
    body_start = html_content.find('<body>')
    if body_start != -1:
        plot_div_class_pos = html_content.find('class="plotly-graph-div"', body_start)
        if plot_div_class_pos != -1:
            div_start = html_content.rfind('<div', body_start, plot_div_class_pos)
            if div_start != -1:
                full_html = html_content[:div_start] + writeup_html + html_content[div_start:]
            else:
                full_html = html_content
        else:
            full_html = html_content
    else:
        full_html = html_content
   
    # Write to file
    if output_file:
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(full_html)
   
    fig.show()
    print(f"Dashboard saved to {output_file} with ARC-AGI write-up rendered at the top.")
# Preview (as requested)
plot_score_over_time(df)
create_dashboard(df)

Dashboard saved to arc_agi_dashboard.html with ARC-AGI write-up rendered at the top.


In [27]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import io
# Neon color palette (bright colors for contrast on dark bg)
NEON_PALETTE = ['#FF00FF', '#00FFFF', '#FFFF00', '#FF0080', '#00FF80', '#8000FF', '#FF8000', '#00FF00']
def prepare_data(df):
    """
    Prepare the DataFrame for plotting.
    """
    df = df.copy()
    if 'Release date' in df.columns:
        df['Release date'] = pd.to_datetime(df['Release date'], errors='coerce')
    # Handle Score, R1, R2, R3: fill NaN with 0
    numeric_cols = ['Score', 'R1', 'R2', 'R3']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            df[col] = df[col].fillna(0)
    # Handle Training compute (FLOP): convert to numeric, leave NaN as NaN (to avoid log issues)
    if 'Training compute (FLOP)' in df.columns:
        df['Training compute (FLOP)'] = pd.to_numeric(df['Training compute (FLOP)'], errors='coerce')
    df['Model version'] = df['Model version'].fillna(df['Name'])
    return df
def plot_score_over_time(df, output_file=None):
    """
    Line plot: Model Score Evolution by Release Date (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Release date', 'Score'])
    df_plot = df_plot.sort_values('Release date')
    fig = px.line(df_plot, x='Release date', y='Score', color='Organization',
                  title='ANLI Score Trends Over Time', markers=True,
                  color_discrete_sequence=NEON_PALETTE)
    fig.update_xaxes(type='date')
    fig.update_layout(template='plotly_dark', font=dict(color='white')) # Dark theme + white text
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_bar_by_org(df, output_file=None):
    """
    Bar plot: Average Score by Organization (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Score'])
    df_grouped = df_plot.groupby('Organization')['Score'].mean().reset_index()
    fig = px.bar(df_grouped, x='Organization', y='Score',
                 title='Average ANLI Score by Organization',
                 color_discrete_sequence=NEON_PALETTE)
    fig.update_layout(template='plotly_dark', font=dict(color='white')) # Dark theme + white text
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_compute_growth(df, output_file=None):
    """
    Area plot: Cumulative Compute vs. Release Date (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Release date', 'Training compute (FLOP)'])
    df_plot = df_plot[df_plot['Training compute (FLOP)'] > 0]
    df_plot = df_plot.sort_values('Release date')
    fig = px.area(df_plot, x='Release date', y='Training compute (FLOP)', color='Country',
                  title='Model Compute Growth by Country',
                  color_discrete_sequence=NEON_PALETTE)
    fig.update_xaxes(type='date')
    fig.update_layout(template='plotly_dark', font=dict(color='white'))
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_score_distribution(df, output_file=None):
    """
    Box plot: Score Distribution by Country (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Score'])
    fig = px.box(df_plot, x='Country', y='Score', points='outliers',
                 title='Score Distributions by Country',
                 color_discrete_sequence=NEON_PALETTE)
    fig.update_layout(template='plotly_dark', font=dict(color='white'))
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_scaling_curve(df, output_file=None):
    """
    Scatter plot: Score vs. Training Compute (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Score', 'Training compute (FLOP)'])
    df_plot = df_plot[df_plot['Training compute (FLOP)'] > 0]
    fig = px.scatter(df_plot, x='Training compute (FLOP)', y='Score', color='Shots',
                     log_x=True, trendline='ols',
                     title='Score vs. Compute (Scaling Curve)',
                     color_discrete_sequence=NEON_PALETTE)
    fig.update_layout(template='plotly_dark', font=dict(color='white'))
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_r3_bubbles(df, output_file=None):
    """
    Bubble chart: R3 Performance vs. Release Date (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Release date', 'R3', 'Training compute (FLOP)'])
    df_plot = df_plot[df_plot['Training compute (FLOP)'] > 0]
    df_plot = df_plot.sort_values('Release date')
    fig = px.scatter(df_plot, x='Release date', y='R3', size='Training compute (FLOP)',
                     color='Organization', hover_name='Name',
                     title='R3 Scores Over Time (Bubble = Compute)',
                     color_discrete_sequence=NEON_PALETTE)
    fig.update_xaxes(type='date')
    fig.update_layout(template='plotly_dark', font=dict(color='white'))
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_correlation_heatmap(df, output_file=None):
    """
    Heatmap: Correlation Matrix (dark theme with neon-inspired scale).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['R1', 'R2', 'R3', 'Score'])
    corr_df = df_plot[['R1', 'R2', 'R3', 'Score']].corr()
    fig = go.Figure(data=go.Heatmap(z=corr_df.values, x=corr_df.columns, y=corr_df.index,
                                    colorscale='Plasma'))
    fig.update_layout(template='plotly_dark', title='ANLI Metrics Correlation Heatmap', font=dict(color='white'))
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_performance_hierarchy(df, output_file=None):
    """
    Sunburst plot: Hierarchical Breakdown (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Score', 'Country', 'Organization'])
    org_scores = df_plot.groupby(['Country', 'Organization'])['Score'].mean().reset_index()
    fig = px.sunburst(org_scores, path=['Country', 'Organization'], values='Score',
                      title='Performance Hierarchy: Country > Org',
                      color_discrete_sequence=NEON_PALETTE)
    fig.update_layout(template='plotly_dark', font=dict(color='white'))
    if output_file:
        fig.write_html(output_file)
    fig.show()
def create_dashboard(df, output_file='anli_dashboard.html'):
    """
    Combined dashboard (dark theme) with Markdown write-up inserted at the top.
    """
    df_plot = prepare_data(df)
    fig = make_subplots(rows=2, cols=2,
                        subplot_titles=('Score Over Time', 'Scaling Curve',
                                        'Bar by Org', 'Correlation Heatmap'),
                        specs=[
                            [{"secondary_y": False}, {"secondary_y": False}],
                            [{"secondary_y": False}, {"type": "heatmap"}]
                        ])
    # Line plot
    df_line = df_plot.dropna(subset=['Release date', 'Score']).sort_values('Release date')
    line_fig = px.line(df_line, x='Release date', y='Score', color='Organization', markers=True,
                       color_discrete_sequence=NEON_PALETTE)
    for trace in line_fig.data:
        fig.add_trace(trace, row=1, col=1)
    # Scatter plot
    df_scatter = df_plot.dropna(subset=['Score', 'Training compute (FLOP)'])
    df_scatter = df_scatter[df_scatter['Training compute (FLOP)'] > 0]
    scatter_fig = px.scatter(df_scatter, x='Training compute (FLOP)', y='Score', log_x=True,
                             color_discrete_sequence=NEON_PALETTE)
    for trace in scatter_fig.data:
        fig.add_trace(trace, row=1, col=2)
    # Bar plot: Average Score by Org
    df_bar = df_plot.dropna(subset=['Score'])
    df_grouped_bar = df_bar.groupby('Organization')['Score'].mean().reset_index()
    bar_fig = px.bar(df_grouped_bar, x='Organization', y='Score',
                     color_discrete_sequence=NEON_PALETTE)
    for trace in bar_fig.data:
        fig.add_trace(trace, row=2, col=1)
    # Heatmap
    df_heatmap = df_plot.dropna(subset=['R1', 'R2', 'R3', 'Score'])
    corr_df = df_heatmap[['R1', 'R2', 'R3', 'Score']].corr()
    heatmap = go.Heatmap(z=corr_df.values, x=corr_df.columns, y=corr_df.index, colorscale='Plasma')
    fig.add_trace(heatmap, row=2, col=2)
    fig.update_layout(height=800, title_text="ANLI Model Performance Dashboard (Dark Neon Theme)",
                      template='plotly_dark', font=dict(color='white'))
   
    # Generate HTML string
    html_content = fig.to_html(full_html=True, include_plotlyjs='cdn')
   
    # Markdown write-up as HTML (dark-themed)
    writeup_html = '''
    <div style="padding: 20px; background-color: #1f1f1f; color: #ffffff; border-bottom: 2px solid #333333; font-family: Arial, sans-serif;">
        <h2 style="color: #00FFFF; margin-bottom: 15px;">Adversarial Natural Language Inference (ANLI)</h2>
        <p style="line-height: 1.6; font-size: 14px;">
            The Adversarial NLI (ANLI) dataset is a crowdsourced benchmark for natural language inference that emphasizes model robustness.
            Unlike traditional NLI datasets like SNLI or MNLI, ANLI involves human annotators iteratively creating deceptive examples designed to fool existing models,
            simulating real-world adversarial attacks. This adversarial process results in a more challenging evaluation for understanding inference in diverse, tricky scenarios.
            The dataset is divided into three rounds (R1, R2, R3) of increasing difficulty, with overall <strong>Score</strong> reflecting combined performance.
        </p>
        <p style="line-height: 1.6; font-size: 14px; margin-top: 10px;">
            ANLI highlights gaps in model generalization and has become a key metric for assessing advanced LLMs on reasoning under deception.
        </p>
    </div>
    '''
   
    # Improved insertion: after scripts, before plot div
    body_start = html_content.find('<body>')
    if body_start != -1:
        plot_div_class_pos = html_content.find('class="plotly-graph-div"', body_start)
        if plot_div_class_pos != -1:
            div_start = html_content.rfind('<div', body_start, plot_div_class_pos)
            if div_start != -1:
                full_html = html_content[:div_start] + writeup_html + html_content[div_start:]
            else:
                full_html = html_content
        else:
            full_html = html_content
    else:
        full_html = html_content
   
    # Write to file
    if output_file:
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(full_html)
   
    fig.show()
    print(f"Dashboard saved to {output_file} with ANLI write-up rendered at the top.")
# Preview (as requested)
plot_score_over_time(df)
create_dashboard(df)

Dashboard saved to anli_dashboard.html with ANLI write-up rendered at the top.


In [30]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import io
# Neon color palette (bright colors for contrast on dark bg)
NEON_PALETTE = ['#FF00FF', '#00FFFF', '#FFFF00', '#FF0080', '#00FF80', '#8000FF', '#FF8000', '#00FF00']
def prepare_data(df):
    """
    Prepare the DataFrame for plotting.
    """
    df = df.copy()
    if 'Release date' in df.columns:
        df['Release date'] = pd.to_datetime(df['Release date'], errors='coerce')
    # Handle Percent correct, Percent using correct edit format: fill NaN with 0
    numeric_cols = ['Percent correct', 'Percent using correct edit format']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            df[col] = df[col].fillna(0)
    # Handle Training compute (FLOP): convert to numeric, leave NaN as NaN (to avoid log issues)
    if 'Training compute (FLOP)' in df.columns:
        df['Training compute (FLOP)'] = pd.to_numeric(df['Training compute (FLOP)'], errors='coerce')
    df['Model version'] = df['Model version'].fillna(df['Name']) if 'Name' in df.columns else df['Model version']
    return df
def plot_percent_correct_over_time(df, output_file=None):
    """
    Line plot: Model Percent Correct Evolution by Release Date (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Release date', 'Percent correct'])
    df_plot = df_plot.sort_values('Release date')
    fig = px.line(df_plot, x='Release date', y='Percent correct', color='Organization',
                  title='Aider Polyglot Percent Correct Trends Over Time', markers=True,
                  color_discrete_sequence=NEON_PALETTE)
    fig.update_xaxes(type='date')
    fig.update_layout(template='plotly_dark', font=dict(color='white')) # Dark theme + white text
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_bar_by_org(df, output_file=None):
    """
    Bar plot: Average Percent Correct by Organization (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Percent correct'])
    df_grouped = df_plot.groupby('Organization')['Percent correct'].mean().reset_index()
    fig = px.bar(df_grouped, x='Organization', y='Percent correct',
                 title='Average Aider Polyglot Percent Correct by Organization',
                 color_discrete_sequence=NEON_PALETTE)
    fig.update_layout(template='plotly_dark', font=dict(color='white')) # Dark theme + white text
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_compute_growth(df, output_file=None):
    """
    Area plot: Cumulative Compute vs. Release Date (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Release date', 'Training compute (FLOP)'])
    df_plot = df_plot[df_plot['Training compute (FLOP)'] > 0]
    df_plot = df_plot.sort_values('Release date')
    fig = px.area(df_plot, x='Release date', y='Training compute (FLOP)', color='Country',
                  title='Model Compute Growth by Country',
                  color_discrete_sequence=NEON_PALETTE)
    fig.update_xaxes(type='date')
    fig.update_layout(template='plotly_dark', font=dict(color='white'))
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_percent_correct_distribution(df, output_file=None):
    """
    Box plot: Percent Correct Distribution by Country (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Percent correct'])
    fig = px.box(df_plot, x='Country', y='Percent correct', points='outliers',
                 title='Percent Correct Distributions by Country',
                 color_discrete_sequence=NEON_PALETTE)
    fig.update_layout(template='plotly_dark', font=dict(color='white'))
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_scaling_curve(df, output_file=None):
    """
    Scatter plot: Percent Correct vs. Training Compute (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Percent correct', 'Training compute (FLOP)'])
    df_plot = df_plot[df_plot['Training compute (FLOP)'] > 0]
    fig = px.scatter(df_plot, x='Training compute (FLOP)', y='Percent correct', color='Edit format',
                     log_x=True, trendline='ols',
                     title='Percent Correct vs. Compute (Scaling Curve)',
                     color_discrete_sequence=NEON_PALETTE)
    fig.update_layout(template='plotly_dark', font=dict(color='white'))
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_edit_format_bubbles(df, output_file=None):
    """
    Bubble chart: Percent Using Correct Edit Format vs. Release Date (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Release date', 'Percent using correct edit format', 'Training compute (FLOP)'])
    df_plot = df_plot[df_plot['Training compute (FLOP)'] > 0]
    df_plot = df_plot.sort_values('Release date')
    fig = px.scatter(df_plot, x='Release date', y='Percent using correct edit format', size='Training compute (FLOP)',
                     color='Organization', hover_name='Model version',
                     title='Percent Using Correct Edit Format Over Time (Bubble = Compute)',
                     color_discrete_sequence=NEON_PALETTE)
    fig.update_xaxes(type='date')
    fig.update_layout(template='plotly_dark', font=dict(color='white'))
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_correlation_heatmap(df, output_file=None):
    """
    Heatmap: Correlation Matrix (dark theme with neon-inspired scale).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Percent correct', 'Percent using correct edit format'])
    corr_cols = ['Percent correct', 'Percent using correct edit format']
    corr_df = df_plot[corr_cols].corr()
    fig = go.Figure(data=go.Heatmap(z=corr_df.values, x=corr_df.columns, y=corr_df.index,
                                    colorscale='Plasma'))
    fig.update_layout(template='plotly_dark', title='Aider Polyglot Metrics Correlation Heatmap', font=dict(color='white'))
    if output_file:
        fig.write_html(output_file)
    fig.show()
def plot_performance_hierarchy(df, output_file=None):
    """
    Sunburst plot: Hierarchical Breakdown (dark theme).
    """
    df_plot = prepare_data(df)
    df_plot = df_plot.dropna(subset=['Percent correct', 'Country', 'Organization'])
    org_scores = df_plot.groupby(['Country', 'Organization'])['Percent correct'].mean().reset_index()
    fig = px.sunburst(org_scores, path=['Country', 'Organization'], values='Percent correct',
                      title='Performance Hierarchy: Country > Org',
                      color_discrete_sequence=NEON_PALETTE)
    fig.update_layout(template='plotly_dark', font=dict(color='white'))
    if output_file:
        fig.write_html(output_file)
    fig.show()
def create_dashboard(df, output_file='aider_polyglot_dashboard.html'):
    """
    Combined dashboard (dark theme) with Markdown write-up inserted at the top.
    """
    df_plot = prepare_data(df)
    fig = make_subplots(rows=2, cols=2,
                        subplot_titles=('Percent Correct Over Time', 'Scaling Curve',
                                        'Bar by Org', 'Correlation Heatmap'),
                        specs=[
                            [{"secondary_y": False}, {"secondary_y": False}],
                            [{"secondary_y": False}, {"type": "heatmap"}]
                        ])
    # Line plot
    df_line = df_plot.dropna(subset=['Release date', 'Percent correct']).sort_values('Release date')
    line_fig = px.line(df_line, x='Release date', y='Percent correct', color='Organization', markers=True,
                       color_discrete_sequence=NEON_PALETTE)
    for trace in line_fig.data:
        fig.add_trace(trace, row=1, col=1)
    # Scatter plot
    df_scatter = df_plot.dropna(subset=['Percent correct', 'Training compute (FLOP)'])
    df_scatter = df_scatter[df_scatter['Training compute (FLOP)'] > 0]
    scatter_fig = px.scatter(df_scatter, x='Training compute (FLOP)', y='Percent correct', log_x=True,
                             color_discrete_sequence=NEON_PALETTE)
    for trace in scatter_fig.data:
        fig.add_trace(trace, row=1, col=2)
    # Bar plot: Average Percent Correct by Org
    df_bar = df_plot.dropna(subset=['Percent correct'])
    df_grouped_bar = df_bar.groupby('Organization')['Percent correct'].mean().reset_index()
    bar_fig = px.bar(df_grouped_bar, x='Organization', y='Percent correct',
                     color_discrete_sequence=NEON_PALETTE)
    for trace in bar_fig.data:
        fig.add_trace(trace, row=2, col=1)
    # Heatmap
    df_heatmap = df_plot.dropna(subset=['Percent correct', 'Percent using correct edit format'])
    corr_cols = ['Percent correct', 'Percent using correct edit format']
    corr_df = df_heatmap[corr_cols].corr()
    heatmap = go.Heatmap(z=corr_df.values, x=corr_df.columns, y=corr_df.index, colorscale='Plasma')
    fig.add_trace(heatmap, row=2, col=2)
    fig.update_layout(height=800, title_text="Aider Polyglot Model Performance Dashboard (Dark Neon Theme)",
                      template='plotly_dark', font=dict(color='white'))
   
    # Generate HTML string
    html_content = fig.to_html(full_html=True, include_plotlyjs='cdn')
   
    # Markdown write-up as HTML (dark-themed)
    writeup_html = '''
    <div style="padding: 20px; background-color: #1f1f1f; color: #ffffff; border-bottom: 2px solid #333333; font-family: Arial, sans-serif;">
        <h2 style="color: #00FFFF; margin-bottom: 15px;">Aider Polyglot Benchmark</h2>
        <p style="line-height: 1.6; font-size: 14px;">
            The Aider Polyglot benchmark evaluates large language models (LLMs) on their ability to assist in code editing across multiple programming languages.
            It uses 225 challenging coding exercises from Exercism in C++, Go, Java, JavaScript, Python, and Rust. Models are tasked with editing code to pass test suites,
            simulating real-world AI-assisted development scenarios.
        </p>
        <p style="line-height: 1.6; font-size: 14px; margin-top: 10px;">
            The <strong>Percent correct</strong> measures the proportion of exercises solved successfully, while <strong>Percent using correct edit format</strong> assesses adherence to proper edit instructions.
            This benchmark highlights models' polyglot coding proficiency and editing precision under diverse linguistic constraints.
        </p>
    </div>
    '''
   
    # Improved insertion: after scripts, before plot div
    body_start = html_content.find('<body>')
    if body_start != -1:
        plot_div_class_pos = html_content.find('class="plotly-graph-div"', body_start)
        if plot_div_class_pos != -1:
            div_start = html_content.rfind('<div', body_start, plot_div_class_pos)
            if div_start != -1:
                full_html = html_content[:div_start] + writeup_html + html_content[div_start:]
            else:
                full_html = html_content
        else:
            full_html = html_content
    else:
        full_html = html_content
   
    # Write to file
    if output_file:
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(full_html)
   
    fig.show()
    print(f"Dashboard saved to {output_file} with Aider Polyglot write-up rendered at the top.")
# Preview (as requested)
plot_percent_correct_over_time(df)
create_dashboard(df)

Dashboard saved to aider_polyglot_dashboard.html with Aider Polyglot write-up rendered at the top.
